In [1]:
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import numpy as np
import scipy
import time
from scipy import signal
# import cvxpy
from scipy.sparse.linalg import svds
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
from scipy.interpolate import UnivariateSpline
from inspect import currentframe
from datetime import datetime
from scipy.linalg import toeplitz
from scipy import optimize
from scipy.stats import rv_continuous
from scipy.stats import gaussian_kde
import random
import sys
import csv
import itertools
import os
import datetime
import copy
sys.setrecursionlimit(2000)
sSaveDataPath = "C:\\Users\\benze\\Documents\\YNU\\市毛研究室\\M2森本\\AF_method\\output\\"
dt_now = datetime.datetime.now()
dirname = str(datetime.datetime.now().year) + str(datetime.datetime.now().month) + str(datetime.datetime.now().day)
date = dirname
os.makedirs(dirname, exist_ok=True)
###### Config ##################################################################
mode_output = 1
trial = 1
iRange = 0 #rangefftでのレンジの選択
peakOffset = 0#
S_AF = 6
thr_numofsource = 0.8

saveData = 0

# threshold
fft_thr = -3.
false_thr = 90.#推定が成功したかどうかを見分ける値

# range fix mode
fixMode = 1 #0の時はdetected bin の中のirange番目のビン，1の時は全体のfix_bin番目をとる
fix_bin = 84 #fftsample=512の時は84，fftsample=256の時は42,fftsample=4の時は1
S_bin = 0

###### antenna location ######
# 3x2 MIMO ULA
txArray = np.array([[0,1,0],[0,3,0],[0,5,0]])
rxArray = np.array([[0,1,0],[0,2,0]])

###### parameter for graph ##########
paramList = np.array([20.0],dtype=np.float) # SNR

###### DOA (azimuth,elevation) ######
doa = np.array([[-14,0],[66,0]],dtype=float)

###### distance of sources ######
dist = np.array([1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.],dtype=np.float)

###### amplitude of sources ######
amp = np.array([1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,1.],dtype=np.float)

###### SNR (about the first source)######
SNR = -20.0

##### RCS (about the last source)########
rcs = 50

###### Snapshots ######
numOfSample = 128
#fftSample = numOfSample
fftSample = 512
# paramList = np.array([256])

###### block ######
numOfBlock = 4

###### sampling frequency ######
#sampleFreq = 5.5E6
sampleFreq = 2.75E6
fc = 79.0E9

###### antenna distance ######
lam = 3.0E8/fc
antDis = lam*0.5
df = 3.16E9/(256/5.5E6)

###### beamspace ######
azAngleArray = np.arange(-90,91,0.1,dtype=float)
elAngleArray = np.arange(-90,91,0.5,dtype=float)
TR=np.arange(1,1001,dtype=float)

###### augment array index ######
numOfAugArray = 6
augNum = np.array([12,13,14,8,9,0,1,2,11,7,3],dtype=int)
subArrayNum = np.array([0,1,2,3,4,5],dtype=int)

###### other values ######
numOfTx = txArray.shape[0]
numOfRx = rxArray.shape[0]
numOfRxData = numOfTx*numOfRx
numOfSource = doa.shape[0]
azAngleLen = len(azAngleArray)
elAngleLen = len(elAngleArray)
dt = 1/sampleFreq
dtArray = np.arange(numOfSample)*dt
half_ratio = lam*0.5/antDis

def checks(*args):
    names = {id(v):k for k,v in currentframe().f_back.f_locals.items()}
    print(', '.join(names.get(id(arg),'???')+' =\n'+repr(arg) for arg in args))

# generate simulated signal
def makeSignal():
    # sample_s = 1000
    sample_s = numOfSample
    # reference signal
    dataArray = np.zeros(numOfBlock*numOfRxData*numOfSample,dtype=complex).reshape(numOfBlock,numOfRxData,numOfSample)
    autoArray = np.zeros(numOfSource*sample_s,dtype=complex).reshape(numOfSource,sample_s)
    psCovArray = np.zeros(numOfSource*numOfSource,dtype=complex).reshape(numOfSource,numOfSource)
    phaseRand = np.random.rand(numOfBlock)-0.5
    phase = np.exp(1j*np.pi*(phaseRand))
    #print(phaseRand*180)
    for iter in range(numOfSource):
        thetaAz = doa[iter,0]
        thetaEl = doa[iter,1]
        R = dist[iter]
        [testArray,testArray_s] = ArrayAntTestSig3D(thetaAz,thetaEl,amp[iter],R,phase,sample_s)
        dataArray += testArray
        autoArray[iter,:] = testArray_s

    freq = 2*dist[0:numOfSource]*BW/(Tm*3E8)
    print("beat frequency=\n{0}\n".format(freq))
    print("beat interval=\n{0}\n".format(1.0/freq))

    # signal covariance (≠ correlation)
    psCovArray += autoArray.dot(np.conjugate(autoArray).T)/float(sample_s)
    print("signal covariance =\n{0}\n".format(psCovArray))
    print("signal covariance (Abs)=\n{0}\n".format(np.abs(psCovArray)))
    print("signal covariance (Arg)=\n{0}\n".format(np.arctan2(np.imag(psCovArray),np.real(psCovArray))*180.0/np.pi))
    if doa.shape[0] != 1:
        cc = psCovArray[0,1]
    else:
        cc = 0.

    # signal power
    ps_array = np.diag(psCovArray)
    ps = np.real(ps_array[0])
    print("signal power=\n{0}\n".format(ps_array))

    # crlb2 = CRLB2(psCovArray)
    A_crlb = np.zeros(numOfRxData*numOfSource,dtype=np.complex).reshape(numOfRxData,numOfSource)
    A_crlb_H = []
    for iter in range(numOfSource):
        A_crlb[:,iter] = ArrayWeight(doa[iter,0],doa[iter,1])
    A_crlb_H = np.conj(A_crlb.T)
    PN = np.eye(numOfRxData) - A_crlb.dot(np.linalg.inv(A_crlb_H.dot(A_crlb))).dot(A_crlb_H)
    # 1D-DOA
    # B_crlb = (1j*2*np.pi*antDis/lam*np.sin(np.deg2rad(doa[:,0]))[:,np.newaxis]).T*np.exp(np.outer(np.arange(numOfRxData),-1j*2*np.pi*antDis/lam*cos(np.deg2rad(doa[:,0]))))*np.arange(numOfRxData)[:,np.newaxis]
    # 1D-DOA(azimuth)
    B_crlb_0 = np.exp(np.outer(np.arange(numOfRxData),1j*2.*np.pi*antDis/lam*sin(np.deg2rad(doa[:,0]))))*np.arange(numOfRxData)[:,np.newaxis]
    B_crlb_00 = (1j*2.*np.pi*antDis/lam*np.cos(np.deg2rad(doa[:,0]))[:,np.newaxis]).T
    B_crlb = B_crlb_00*B_crlb_0
    # 2D-DOA rev2 (for orthogonal Tx-Rx location)
    # B_crlb_y = (1j*2*np.pi/lam*antDis*(np.cos(np.deg2rad(doa[:,0]))*np.cos(np.deg2rad(doa[:,1])))[:,np.newaxis]).T*np.exp(np.outer(np.arange(numOfRx),1j*2*np.pi*sin(np.deg2rad(doa[:,0]))*cos(np.deg2rad(doa[:,1]))*antDis/lam))*np.arange(numOfRx)[:,np.newaxis]
    # B_crlb_z = (1j*2*np.pi/lam*antDis*np.cos(np.deg2rad(doa[:,1]))[:,np.newaxis]).T*np.exp(np.outer(np.arange(numOfTx),1j*2*np.pi*sin(np.deg2rad(doa[:,1]))*antDis/lam))*np.arange(numOfTx)[:,np.newaxis]
    # B_crlb = np.zeros(numOfRxData*numOfSource,dtype=np.complex).reshape(numOfRxData,numOfSource)
    # for iter in range(numOfSource):
    #     B_crlb[:,iter] = np.kron(B_crlb_z[:,iter],B_crlb_y[:,iter])
    # CRLB Calculation
    B_crlb_H = np.conj(B_crlb.T)
    CC = B_crlb_H.dot(PN.dot(B_crlb))
    try:
        DD = np.linalg.inv(np.real(CC*np.diag(ps_array).T))
        var = np.diag(DD)*ps*(10**(-SNR/10))/(2.*float(numOfSample*numOfBlock))
        crlb2 = np.rad2deg(np.sqrt(var))
        # print("check=\n{0}\n".format(var))
        # print("check=\n{0}\n".format(crlb2))
    except LinAlgError:
        crlb2 = inf
    return dataArray,ps,cc,crlb2

# generate reference signal
def ArrayAntTestSig3D(theta,phai,A,R,randPhase,sample_s):
    # signals
    data = np.zeros(numOfBlock*numOfRxData*numOfSample,dtype=np.complex).reshape(numOfBlock,numOfRxData,numOfSample)
    data_for_S = np.zeros(sample_s,dtype=np.complex)
    #phai+=pnoise

    blockArray = np.arange(numOfBlock)

    # look direction unit vector (derived from polar coordinates system)
    unitV = [np.cos(np.deg2rad(phai))*np.cos(np.deg2rad(theta)),np.cos(np.deg2rad(phai))*np.sin(np.deg2rad(theta)),np.sin(np.deg2rad(phai))]
    # unitV = [np.sin(np.deg2rad(phai))*np.cos(np.deg2rad(theta)),np.sin(np.deg2rad(phai))*np.sin(np.deg2rad(theta)),np.cos(np.deg2rad(phai))]
    xUnitTx = np.array(unitV)
    xUnitRx = np.array(unitV)

    # dot product of position vector and look direction unit vector
    vTx=np.dot(xUnitTx,txArray.T*antDis)
    vRx=np.dot(xUnitRx,rxArray.T*antDis)

    # wave number
    k = 2*np.pi/lam

    # delay time
    tau = 2*R/3E8
    # print(tau)

    # angle-induced phase shift
    pShiftTx = np.exp(1j*k*vTx)
    pShiftRx = np.exp(1j*k*vRx)
    pShiftAngle = np.kron(pShiftTx,pShiftRx)

    # delay-induced phase
    pShiftDelay = np.exp(1j*(-np.pi*df*tau*tau + 2*np.pi*fc*tau))
    print("DOA = [{0} , {1}]".format(theta,phai))
    print("u = {0}\n".format(np.angle(pShiftAngle*pShiftDelay,deg=True)))
    print("phase shift = {0}\n".format(np.angle(pShiftAngle*pShiftDelay,deg=True)))

    for iterBlock in blockArray:
        for iter in range(numOfSample):
        # amplitude * angle-induced phase shift * delay-induced phase shift * delay-induced phase
            # data[iterBlock,:,iter] = A*randPhase[iterBlock]*pShiftAngle*pShiftDelay*np.exp(1j*2*np.pi*df*tau/sampleFreq*iter)
            data[iterBlock,:,iter] = A*pShiftAngle*pShiftDelay*np.exp(1j*2*np.pi*df*tau/sampleFreq*iter)
        # for iterRx in range(numOfRxData):
        #     data[iterBlock,iterRx,:] = data[iterBlock,iterRx,:]*np.blackman(numOfSample)
    for iter in range(sample_s):
        data_for_S[iter] = A*np.exp(1j*2*np.pi*df*tau/sampleFreq*iter)
    return data,data_for_S

# noise
def addNoise(dataArray,ps):
    pn = (10**(-SNR/10))
    sdn = np.sqrt(pn)
    noise_r = np.random.normal(0.0,1,dataArray.shape)
    noise_i = np.random.normal(0.0,1,dataArray.shape)
    noise = (noise_r + 1j*noise_i)*sdn/np.sqrt(2)
    #checks(noise)
    #print(shape(noise))
    sigArray = dataArray + noise
    print("Noise Power=\n{0}\n".format(pn))

    return sigArray,pn

# range FFT
def range_fft(sigArray,fftSample,iRange,peakOffset,fixMode,fix_bin,fft_thr):
    # # initialization
    freqArray = np.fft.fftfreq(fftSample,1/sampleFreq)
    fftInput = np.zeros(numOfRxData*numOfSample*numOfBlock,dtype=np.complex).reshape(numOfBlock,numOfRxData,numOfSample)
    fftInputDS = np.zeros(numOfRxData*fftSample*numOfBlock,dtype=np.complex).reshape(numOfBlock,numOfRxData,fftSample)
    fftInput2 = np.zeros(numOfRxData*numOfSample*numOfBlock,dtype=np.complex).reshape(numOfBlock,numOfRxData,numOfSample)
    fftArray = np.zeros(numOfBlock*numOfRxData*numOfSample,dtype=np.complex).reshape(numOfBlock,numOfRxData,numOfSample)
    #fftArray = np.zeros(numOfBlock*numOfRxData*fftfSample,dtype=np.complex).reshape(numOfBlock,numOfRxData,fftSample)
    fftArrayDS = np.zeros(numOfBlock*numOfRxData*fftSample,dtype=np.complex).reshape(numOfBlock,numOfRxData,fftSample)
    fftArray_mean = np.zeros(numOfRxData*fftSample,dtype=np.complex).reshape(numOfRxData,fftSample)
    fftAmp = np.zeros(numOfRxData*fftSample,dtype=np.float).reshape(numOfRxData,fftSample)
    fftPhase = np.zeros(numOfRxData*fftSample,dtype=np.float).reshape(numOfRxData,fftSample)
    rangeProfile = np.zeros(fftSample,dtype=np.float)

    for iterX in range(numOfRxData):
        for iterBlock in range(numOfBlock):
            if fftSample<numOfSample:
                w1 = signal.blackman(numOfSample)
                w2 = signal.blackman(fftSample)
                # Pattern1
                fftInputDS[iterBlock,iterX,:] = w2*sigArray[iterBlock,iterX,:fftSample]
                fftArrayDS[iterBlock,iterX,:] = np.fft.fft(fftInputDS[iterBlock,iterX,:])

            else:
                w = signal.blackman(numOfSample)
                fftInputDS[iterBlock,iterX,0:numOfSample] = w*sigArray[iterBlock,iterX,:]
                fftArrayDS[iterBlock,iterX,:] = np.fft.fft(fftInputDS[iterBlock,iterX,:])

        fftArray_mean[iterX,:] = np.mean(fftArrayDS[:,iterX,:],axis=0)
        fftAmp[iterX,:] = 10*log10(np.abs(fftArray_mean[iterX,:]))
        fftPhase[iterX,:] = np.rad2deg(np.arctan(np.imag(fftArray_mean[iterX,:])/np.real(fftArray_mean[iterX,:])))
        rangeProfile[:] += fftAmp[iterX,:]

    # # peak detection
    rangeProfile /= numOfRxData
    peakMax = np.max(rangeProfile)
    peakID = np.array(signal.argrelmax(rangeProfile[0:int(fftSample/2)]))
    peakID = peakID[rangeProfile[peakID] > peakMax + fft_thr] + peakOffset
    numOfPeak = len(peakID)
    peakFreq = peakID*sampleFreq/fftSample
    peakRange = peakFreq*Tm*3E8/2/BW
    print("detected bin=\n{0}\n".format(peakID))
    print("detected range=\n{0}\n".format(peakRange))
    global S_bin
    if not S_bin ==0:
        select = S_bin
    elif fixMode == 1:
        select = fix_bin
    elif fixMode ==0:
        select = peakID[iRange]
    S_bin = select
    print("selected range=\n{0}\n".format(select*sampleFreq/fftSample*Tm*3E8/2/BW))
    return fftArray[:,:,select],fftArray_mean[:,select],fftArray,fftAmp,fftPhase,rangeProfile,peakID,fftArrayDS[:,:,select]


# array mode vector
def ArrayWeight(theta,phai):
    # unit vector derived from poplar coordinates system
    unitV = [np.cos(np.deg2rad(phai))*np.cos(np.deg2rad(theta)),np.cos(np.deg2rad(phai))*np.sin(np.deg2rad(theta)),np.sin(np.deg2rad(phai))]
    # unitV = [np.sin(np.deg2rad(phai))*np.cos(np.deg2rad(theta)),np.sin(np.deg2rad(phai))*np.sin(np.deg2rad(theta)),np.cos(np.deg2rad(phai))]
    unitArray = np.array(unitV)

    xUnitTx = np.array(unitV)
    xUnitRx = np.array(unitV)

    # dot product of position vector and look direction unit vector
    vTx=np.dot(xUnitTx,txArray.T*antDis)
    vRx=np.dot(xUnitRx,rxArray.T*antDis)
    # wave number
    k = 2*np.pi/lam

    pShiftTx = np.exp(1j*k*vTx)
    pShiftRx = np.exp(1j*k*vRx)
    pShiftAngle = np.kron(pShiftTx,pShiftRx)

    return 1/np.sqrt(numOfRxData)*pShiftAngle

# monic polynomial roots
def roots(vec,cmplx=True):
    # f(x) = vec[0] + vec[1]x + vec[2]x^2 + ... + vec[n-1]x^(n-1) + x^n
    dim =len(vec)
    if cmplx:
        A = np.zeros((dim,dim),dtype=complex)
    else:
        A = np.zeros((dim,dim))
    A[np.arange(dim-1),1+np.arange(dim-1)] = 1
    #checks(A)
    A[-1,:] = -vec
    #checks(A)
    ans,vec = np.linalg.eig(A+0.000001)
    return ans

def spurCoefficient():
    coef1=np.zeros(4*2).reshape(4,2)    #(6,1)
    coef2=np.zeros(6*2).reshape(6,2)    #(5,1),(6,2)
    coef3=np.zeros(8*2).reshape(8,2)    #(4,1),(5,2),(6,3)
    coef4=np.zeros(10*2).reshape(10,2)  #(3,1),(4,2),(5,3),(6,4)
    coef5=np.zeros(12*2).reshape(12,2)  #(2,1),(3,2),(4,3),(5,4),(6,5)
    coef6=np.zeros(14*2).reshape(14,2)  #(1,1),...,(6,6)

    coef1[:,0] = np.array([5,0,6,-1]).T
    coef1[:,1] = np.array([0,5,-1,6]).T

    for iterM in range(3):
        coef2[2*iterM:2*iterM+2,:] = 4+iterM
        coef2[2*iterM,1] = -iterM
        coef2[2*iterM+1,0] = -iterM
    for iterM in range(4):
        coef3[2*iterM:2*iterM+2,:] = 3+iterM
        coef3[2*iterM,1] = -iterM
        coef3[2*iterM+1,0] = -iterM
    for iterM in range(5):
        coef4[2*iterM:2*iterM+2,:] = 2+iterM
        coef4[2*iterM,1] = -iterM
        coef4[2*iterM+1,0] = -iterM
    for iterM in range(6):
        coef5[2*iterM:2*iterM+2,:] = 1+iterM
        coef5[2*iterM,1] = -iterM
        coef5[2*iterM+1,0] = -iterM
    for iterM in range(7):
        coef6[2*iterM:2*iterM+2,:] = 0+iterM
        coef6[2*iterM,1] = -iterM
        coef6[2*iterM+1,0] = -iterM
    #checks(coef3)
    l1 = coef1.shape[0]*coef2.shape[0]
    l2 = coef2.shape[0]*coef3.shape[0]
    l3 = coef3.shape[0]*coef4.shape[0]
    l4 = coef4.shape[0]*coef5.shape[0]
    l5 = coef5.shape[0]*coef6.shape[0]

    coef12 = np.zeros(l1*2).reshape(l1,2)
    coef23 = np.zeros(l2*2).reshape(l2,2)
    coef34 = np.zeros(l3*2).reshape(l3,2)
    coef45 = np.zeros(l4*2).reshape(l4,2)
    coef56 = np.zeros(l5*2).reshape(l5,2)

    for iterN in range(4):
        for iterM in range(6):
            coef12[iterN*6+iterM,:]=coef2[iterM,:] - coef1[iterN,:]
    for iterN in range(6):
        for iterM in range(8):
            coef23[iterN*8+iterM,:]=coef3[iterM,:] - coef2[iterN,:]
    for iterN in range(8):
        for iterM in range(10):
            coef34[iterN*10+iterM,:]=coef4[iterM,:] - coef3[iterN,:]
    for iterN in range(10):
        for iterM in range(12):
            coef45[iterN*12+iterM,:]=coef5[iterM,:] - coef4[iterN,:]
    for iterN in range(12):
        for iterM in range(14):
            coef56[iterN*14+iterM,:]=coef6[iterM,:] - coef5[iterN,:]
    """
    Co2,coefFreq = np.unique(coef12,return_counts=True,axis=0)
    Co2,coefFreq = np.unique(coef23,return_counts=True,axis=0)
    Co2,coefFreq = np.unique(coef34,return_counts=True,axis=0)
    Co2,coefFreq = np.unique(coef45,return_counts=True,axis=0)
    Co2,coefFreq = np.unique(coef56,return_counts=True,axis=0)
    """
    Co = np.concatenate([coef12,coef23,coef34,coef45,coef56])
    Co2,coefFreq = np.unique(Co,return_counts=True,axis=0)
    return Co2

# Make KR prodcuts Expanded Signals by Single Snapshot Signal
def makeKRsig(array):
    arrayDim = array.ndim
    print('Shape of Original Signal = {0}'.format(array.shape))
    if arrayDim == 1:
        array_length = array.shape[0]
        corrArray = np.outer(array[:],np.conjugate(array[:]))
        checks(corrArray.shape)

        KRsig = np.zeros(11,dtype=np.complex)
        corrArray2 = corrArray[1:,1:]
        #KRsig = corrArray[:,:].T.reshape(-1,)
        checks(KRsig)
        for iter in range(11):
            iterDiag = iter - 5
            diag = np.diag(corrArray,k=iterDiag)
            KRsig[iter] = np.mean(diag)

        KRsig1 = np.array([corrArray[5,0],corrArray[4,0],corrArray[3,0],corrArray[2,0],\
                            corrArray[1,0],corrArray[0,0],corrArray[0,1],corrArray[0,2],\
                            corrArray[0,3],corrArray[0,4],corrArray[0,5]])
        KRsig2 = np.array([corrArray[5,0],corrArray[5,1],corrArray[4,1],corrArray[3,1],\
                            corrArray[2,1],corrArray[1,1],corrArray[1,2],corrArray[1,3],\
                            corrArray[1,4],corrArray[1,5],corrArray[0,5]])
        KRsig3 = np.array([corrArray[5,0],corrArray[5,1],corrArray[5,2],corrArray[4,2],\
                            corrArray[3,2],corrArray[2,2],corrArray[2,3],corrArray[2,4],\
                            corrArray[2,5],corrArray[1,5],corrArray[0,5]])
        KRsig4 = np.array([corrArray[5,0],corrArray[5,1],corrArray[5,2],corrArray[5,3],\
                            corrArray[4,3],corrArray[3,3],corrArray[3,4],corrArray[3,5],\
                            corrArray[2,5],corrArray[1,5],corrArray[0,5]])
        KRsig5 = np.array([corrArray[5,0],corrArray[5,1],corrArray[5,2],corrArray[5,3],\
                            corrArray[5,4],corrArray[4,4],corrArray[4,5],corrArray[3,5],\
                            corrArray[2,5],corrArray[1,5],corrArray[0,5]])
        KRsig6 = np.array([corrArray[5,0],corrArray[5,1],corrArray[5,2],corrArray[5,3],\
                            corrArray[5,4],corrArray[5,5],corrArray[4,5],corrArray[3,5],\
                            corrArray[2,5],corrArray[1,5],corrArray[0,5]])
        KRsig7 = np.array([corrArray[5,0],corrArray[4,0],corrArray[4,1],corrArray[4,2],\
                            corrArray[4,3],corrArray[4,4],corrArray[3,4],corrArray[2,4],\
                            corrArray[1,4],corrArray[0,4],corrArray[0,5]])
        KRsig8 = np.array([corrArray[5,0],corrArray[4,0],corrArray[3,0],corrArray[3,1],\
                            corrArray[3,2],corrArray[3,3],corrArray[2,3],corrArray[1,3],\
                            corrArray[0,3],corrArray[0,4],corrArray[0,5]])
        KRsig9 = np.array([corrArray[5,0],corrArray[4,0],corrArray[3,0],corrArray[2,0],\
                            corrArray[2,1],corrArray[2,2],corrArray[1,2],corrArray[0,2],\
                            corrArray[0,3],corrArray[0,4],corrArray[0,5]])
        KRsig10 = np.array([corrArray[5,0],corrArray[4,0],corrArray[3,0],corrArray[2,0],\
                            corrArray[1,0],corrArray[1,1],corrArray[0,1],corrArray[0,2],\
                            corrArray[0,3],corrArray[0,4],corrArray[0,5]])
        #KRsig_ave = (KRsig1+KRsig2+KRsig3+KRsig4+KRsig5+KRsig6)/6#+KRsig7+KRsig8+KRsig9+KRsig10)/10

        #KRsig=(KRsig3+KRsig6)/2
        KRsig_all = np.vstack((KRsig1,KRsig2,KRsig3,KRsig4,KRsig5,KRsig6,KRsig7,KRsig8,KRsig9,KRsig10))
        KRsig_ave = np.zeros(11,dtype=np.complex)
        rank_all = np.zeros(10)
        for iterN in range(10):
            array=KRsig_all[iterN,:]
            source_af = S_AF

            # convolution matrix

            array_length = array.shape[0]
            #checks(array_length)
            conv_array = np.zeros((array_length-source_af)*(source_af+1),dtype=np.complex).reshape(array_length-source_af,source_af+1)
            for iter_conv in range(array_length-source_af):
                conv_array[iter_conv,:] = array[iter_conv:iter_conv+source_af+1][-1::-1]
            #checks(conv_array)
            svd_vecs_u, svd_vals, svd_vecs_vh = np.linalg.svd(conv_array)
            #checks(shape(svd_vecs_u))
            #checks((svd_vals))
            #checks(shape(svd_vecs_vh))
            sv = svd_vals
            sv /= max(sv)
            #checks(sv)
            rank_all[iterN] = np.count_nonzero(sv > 0.3)
        checks(rank_all)
        counter = 0
        for iterM in range(10):
            if rank_all[iterM]==min(rank_all):
                #checks(iterM)
                KRsig_ave += KRsig_all[iterM,:]
                counter += 1
        KRsig_ave /= counter
        
        return KRsig[::-1],KRsig1[::-1],KRsig2[::-1],KRsig3[::-1],KRsig4[::-1],KRsig5[::-1],KRsig6[::-1],KRsig7[::-1],KRsig8[::-1],KRsig9[::-1],KRsig10[::-1],KRsig_ave[::-1]

    else:
        snapshot = array.shape[0]
        array_length = array.shape[1]
        numOfFrame = 4
        #numOfFrame = 1
        M = int(snapshot/numOfFrame)

        KRsig = np.zeros(M*11,dtype=np.complex).reshape(M,11)

        for iterM in range(M):
            corrArray = np.zeros(array_length*array_length,dtype=np.complex).reshape(array_length,array_length)
            for iterF in range(numOfFrame):
                corrArray += np.outer(array[numOfFrame*iterM+iterF,:],np.conjugate(array[numOfFrame*iterM+iterF,:]))/numOfFrame

            for iter in range(11):
                iterDiag = iter - 5
                diag = np.diag(corrArray,k=iterDiag)
                KRsig[iterM,iter] = np.mean(diag)

        print('Shape of KRsig = {0}'.format(KRsig.shape))
        return KRsig[:,::-1]

# TLS (annihilating filter method)
def annihilating_1D_nKR(array):
    #source_af = numOfSource
    source_af = S_AF

    # convolution matrix

    array_length = array.shape[0]
    #checks(array_length)
    conv_array = np.zeros((array_length-source_af)*(source_af+1),dtype=np.complex).reshape(array_length-source_af,source_af+1)
    for iter_conv in range(array_length-source_af):
        conv_array[iter_conv,:] = array[iter_conv:iter_conv+source_af+1][-1::-1]
    #checks(conv_array)
    svd_vecs_u, svd_vals, svd_vecs_vh = np.linalg.svd(conv_array)
    #checks(shape(svd_vecs_u))
    checks((svd_vals))
    #checks(shape(svd_vecs_vh))
    sv = np.copy(svd_vals)
    sv /= max(sv)
    checks(sv)
    rank_CM = np.count_nonzero(sv > 0.5)
    checks(rank_CM)

    # DOA estimation
    hs_0 = np.conj(svd_vecs_vh.T)[:,-1][-1::-1]
    #checks(hs_0)
    hs = hs_0/hs_0[-1]
    #checks(hs[:])
    z = roots(hs[:-1])
    checks(z)
    #angle_response(z)


    z2 = np.zeros(source_af,dtype=np.complex)
    for iterZ in range(source_af):
        z2[iterZ] = (1/abs(z[iterZ]))*z[iterZ]
    #angle_response(z2)
    #checks(z2)
    angle = np.array(np.arcsin(np.arctan2(np.imag(z2),np.real(z2))/np.pi)*180/np.pi)[np.newaxis].T
    checks(angle)

    # power estimation
    vand_mat = np.zeros(6*source_af,dtype=np.complex).reshape(6,source_af)
    for iter in range(6):
        vand_mat[iter,:] = z2**iter
    vand_mat_inv = np.linalg.pinv(vand_mat[:])
    #checks(vand_mat)
    checks(vand_mat_inv)
    checks(array)
    p_vec = (np.abs(np.dot(vand_mat_inv,array))/(numOfSample/2.))**2
    checks(p_vec)
    checks(sorted(p_vec, reverse=True))
    p_normal = p_vec/np.max(p_vec)
    num_pow = p_normal
    p_index = np.argsort(p_vec)[-1::-1]
    checks(p_normal[p_index])
    p_normal = p_normal[p_index]
    checks(angle[p_index])
    xangle = angle
    """
    f = open('angle.csv', 'a', newline='')
    writer = csv.writer(f)
    writer.writerows(angle)
    writer.writerows('\n')
    f.close()
    """
    p_normal_min = np.min(p_normal)/np.max(p_vec)
    estimated_angle = angle[p_index]
    angle = angle[p_index[0:numOfSource]]

    root = np.array((np.real(z),np.imag(z))).T
    xrange = np.arange(-1,1.02,0.02)
    y = np.sqrt(1-xrange**2)
    y[-1]=0
    xroot = root
    """
    f = open('root.csv', 'a', newline='')
    writer = csv.writer(f)
    writer.writerows(root)
    writer.writerows('\n')
    f.close()
    """
    print(type(p_vec))
    xpower = p_vec[:, np.newaxis]
    
    f = open('power.csv', 'a', newline='')
    writer = csv.writer(f)
    writer.writerows(p_vec[:, np.newaxis])
    writer.writerows('\n')
    f.close()
    
    plt.figure()
    plt.plot(root[:,0],root[:,1],marker='.',markersize=12,linestyle='None')
    plt.plot(xrange,y,color='black',linestyle=':')
    plt.plot(xrange,-y,color='black',linestyle=':')
    #plt.plot(2,2,color='white',linestyle='None')
    plt.grid(which='major',color='gray',linestyle='--')
    plt.grid(which='minor',color='gray',linestyle=':')
    plt.axis('equal')
    plt.xlabel('Re')
    plt.ylabel('Im')
    #plt.xlim([-2.0,2.0])
    #plt.ylim([-2.0,2.0])
    if len(doa) == 1:
        plt.title('%d'%(doa[0,0]))
    elif len(doa) == 2:
        plt.title('%d,%d'%(doa[0,0],doa[1,0]))
    elif len(doa) == 3:
        plt.title('%d,%d,%d'%(doa[0,0],doa[1,0],doa[2,0]))
    foldername = "単位円到来波%d設定波%dwoKR"%(len(doa),S_AF)
    foldername = dirname+"/"+foldername
    os.makedirs(foldername, exist_ok=True)
    if len(doa) == 1:
        plt.savefig(foldername + "/" + 'root_%d.png'%(doa[0,0]))
    elif len(doa) == 2:
        plt.savefig(foldername + "/" + 'root_%d,%d.png'%(doa[0,0],doa[1,0]))
    elif len(doa) == 3:
        plt.savefig(foldername + "/" + 'root_%d,%d,%d.png'%(doa[0,0],doa[1,0],doa[2,0]))
    plt.show()
    plt.close()
    

    '''
    Angular_ch = np.zeros((S_AF+1)*1810,dtype=np.complex).reshape(1810,S_AF+1)

    for iterNum in range(S_AF+1):
        V = np.zeros((S_AF+1)*(S_AF+1),dtype=np.complex).reshape(S_AF+1,S_AF+1)
        V[:,iterNum] = np.conj(svd_vecs_vh.T[:,iterNum])

        Angular_ch[:,iterNum] = searchBF1D_SAF(V)
    inv = 1/Angular_ch[:,0]
    plt.figure(3,figsize=[8,5])
    plt.plot(azAngleArray,10*np.log10(np.abs(Angular_ch[:,0])/np.max(np.abs(Angular_ch[:,0]))),label='1st')
    plt.plot(azAngleArray,10*np.log10(np.abs(Angular_ch[:,-1])/np.max(np.abs(Angular_ch[:,-1]))),label='last')
    #plt.plot(-azAngleArray,1/(10*np.log10(np.abs(Angular_ch[:,0])/np.max(np.abs(Angular_ch[:,0])))-0.05),label='1st inv')
    #plt.plot(-azAngleArray,10*np.log10((1/np.abs(Angular_ch[:,0])/np.max(1/np.abs(Angular_ch[:,0])))),label='1st inv2')
    #plt.plot(-azAngleArray,10*np.log10(np.abs(inv)/np.max(np.abs(inv))),label='1st inv3')
    #plt.plot(-azAngleArray,10*np.log10(1/((np.abs(Angular_ch[:,0])/np.max(np.abs(Angular_ch[:,0]))-0.01))),label='1st inv4')
    #print(10*np.log10(np.abs(inv)/np.max(np.abs(inv))))

    xlim(-90.0,90.0)
    #ylim(-45.0,1.0)
    plt.xlabel("Azimuth (deg)")
    plt.ylabel("power")
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(20))
    plt.gca().xaxis.set_minor_locator(tick.MultipleLocator(5))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(5))
    plt.grid(which='major',color='gray',linestyle='--')
    plt.grid(which='minor',color='gray',linestyle=':')
    plt.legend(bbox_to_anchor=(1.0, 1), loc='upper left', borderaxespad=0, fontsize=15)
    titlename = "Power spectrum(Beamformer)"
    filename = sSaveDataPath + "K=" + str(numOfSource) + "_Fig_MODE_AngularChalacteristics.png"
    plt.title(titlename)
    #plt.savefig(filename)
    plt.show()
    plt.close

    ##############################################################
    # spurious estimation
    # numOfsspurs
    spurList = np.zeros(10)
    #print(angle0[:,0])
    phaiList = np.pi*np.sin(np.deg2rad(doa[:,0]))
    #print(phaiList)

    # initial phase calculated from distance R and minimum chirp freq (fmin)
    initialPhase = 4*np.pi*(dist[1]-dist[0])*fc/3.0E8

    # spurious list (if phase)
    spurList[0] = 3*phaiList[0]-2*phaiList[1] #+ initialPhase
    spurList[1] = 3*phaiList[1]-2*phaiList[0] #- initialPhase
    spurList[2] = 4*phaiList[0]-3*phaiList[1] #+ initialPhase
    spurList[3] = 4*phaiList[1]-3*phaiList[0] #- initialPhase
    spurList[4] = 2*phaiList[0] - phaiList[1] #+ initialPhase
    spurList[5] = 2*phaiList[1] - phaiList[0] #- initialPhase
    spurList[6] = phaiList[0] #+initialPhase
    spurList[7] = phaiList[1] #-initialPhase
    spurList[8] = 5 * phaiList[0] - 4 * phaiList[1]  # + initialPhase
    spurList[9] = 5 * phaiList[0] - 4 * phaiList[1]  # + initialPhase
    spurList*=1
    #print(spurList)

    # to keep angle in range from -pi to pi (wrap)
    spurList = (spurList+np.pi)%(2*np.pi)-np.pi
    #print(spurList)

    # to convert IF phase (phai) to arrival angle (theta)
    thetaList = np.rad2deg(np.arcsin(spurList/np.pi))
    #print(thetaList)


    plt.figure()
    for iterN in range(S_AF):
        plt.bar(estimated_angle[iterN],p_normal[iterN])
    plt.vlines(thetaList,0,1,color="red",linestyles="dashed",alpha=0.3)
    plt.vlines(doa[:,0],0,1,color="blue",linestyles="dashed",alpha=0.3)
    plt.yscale('log')
    plt.xlim([-90,90])
    plt.ylim(0.001,1.2)
    plt.grid(which='major',color='gray',linestyle=':')
    plt.grid(which='minor',color='gray',linestyle=':')
    plt.ylabel('Normalized Power',fontsize=15)
    plt.xlabel('Angle [deg.]',fontsize=15)
    #name = sSaveDataPath + date + '_NormPower_K=' + str(numOfSource) + '_' + str(SNR) + 'dB_P.png'
    #plt.savefig(name,bbox_inches='tight',pad_inches=0.0)

    #plt.show()
    #plt.close
    '''
    #return angle,conv_array,p_normal_min
    return angle,conv_array,np.squeeze(estimated_angle),p_normal,svd_vals,xroot,xangle,xpower,num_pow,svd_vecs_vh,svd_vecs_u,z2



# (AF法に不連続除去適用)
def annihilating_1D_DR(array):
    #source_af = numOfSource
    source_af = S_AF

    # convolution matrix
    array_length = array.shape[0]
    #checks(array_length)
    #設定波数3の場合
    if source_af == 3:
        conv_array = np.zeros((array_length-source_af-2)*(source_af+1),dtype=np.complex).reshape(array_length-source_af-2,source_af+1)
        for iter_conv in range(array_length-source_af-2):
            if iter_conv < 3:
                conv_array[iter_conv,:] = array[iter_conv:iter_conv+source_af+1][-1::-1]
            else:
                conv_array[iter_conv,:] = array[iter_conv+2:iter_conv+2+source_af+1][-1::-1]

    #checks(conv_array)
    svd_vecs_u, svd_vals, svd_vecs_vh = np.linalg.svd(conv_array)
    #checks(shape(svd_vecs_u))
    checks((svd_vals))
    #checks(shape(svd_vecs_vh))
    sv = np.copy(svd_vals)
    sv /= max(sv)
    checks(sv)
    rank_CM = np.count_nonzero(sv > 0.5)
    checks(rank_CM)

    # DOA estimation
    hs_0 = np.conj(svd_vecs_vh.T)[:,-1][-1::-1]
    #checks(hs_0)
    hs = hs_0/hs_0[-1]
    #checks(hs[:])
    z = roots(hs[:-1])
    checks(z)
    #angle_response(z)

    z2 = np.zeros(source_af,dtype=np.complex)
    for iterZ in range(source_af):
        z2[iterZ] = (1/abs(z[iterZ]))*z[iterZ]
    #angle_response(z2)
    #checks(z2)
    angle = np.array(np.arcsin(np.arctan2(np.imag(z2),np.real(z2))/np.pi)*180/np.pi)[np.newaxis].T
    checks(angle)

    #filename = sSaveDataPath + "txt_annihilating_parameter.txt"
    #fout = open(filename,'w')
    # z_abs = np.abs(z)
    #fout.write("%s"%hs)
    # fout.write("\n")
    # fout.write("%s"%z_abs)
    # fout.write("\n")
    # fout.write("%s"%angle)
    # fout.write("\n")
    # fout.close()

    # power estimation
    #vand_mat = np.zeros(11*source_af,dtype=np.complex).reshape(11,source_af)
    #for iter in range(11):
    #    vand_mat[iter,:] = z**iter
    #vand_mat_inv = np.linalg.pinv(vand_mat[:])
    #checks(vand_mat)
    # checks(vand_mat_inv)
    #p_vec = (np.abs(np.dot(vand_mat_inv,array))/(numOfSample/2.))**2
    #p_normal = p_vec/np.max(p_vec)
    #p_index = np.argsort(p_vec)[-1::-1]
    #checks(p_vec[p_index])
    #checks(p_normal[p_index])
    #checks(angle[p_index])
    #p_normal_min = np.min(p_normal)/np.max(p_vec)
    #estimated_angle = angle[p_index]
    #angle = angle[p_index[0:numOfSource]]

    # power estimation
    vand_mat = np.zeros(11*source_af,dtype=np.complex).reshape(11,source_af)
    for iter in range(11):
        vand_mat[iter,:] = z2**iter
    vand_mat_inv = np.linalg.pinv(vand_mat[:])
    #checks(vand_mat)
    # checks(vand_mat_inv)
    p_vec = (np.abs(np.dot(vand_mat_inv,array))/(numOfSample/2.))**2
    checks(p_vec)
    checks(sorted(p_vec, reverse=True))
    p_normal = p_vec/np.max(p_vec)
    num_pow = p_normal
    p_index = np.argsort(p_vec)[-1::-1]
    checks(p_normal[p_index])
    p_normal = p_normal[p_index]
    checks(angle[p_index])
    xangle = angle
    """
    f = open('angle.csv', 'a', newline='')
    writer = csv.writer(f)
    writer.writerows(angle)
    writer.writerows('\n')
    f.close()
    """
    p_normal_min = np.min(p_normal)/np.max(p_vec)
    estimated_angle = angle[p_index]
    angle = angle[p_index[0:numOfSource]]

    root = np.array((np.real(z),np.imag(z))).T
    xrange = np.arange(-1,1.02,0.02)
    y = np.sqrt(1-np.around(xrange,10)**2)
    y[-1]=0
    xroot = root
    """
    f = open('root.csv', 'a', newline='')
    writer = csv.writer(f)
    writer.writerows(root)
    writer.writerows('\n')
    f.close()
    """
    print(type(p_vec))
    xpower = p_vec[:, np.newaxis]
    
    f = open('power.csv', 'a', newline='')
    writer = csv.writer(f)
    writer.writerows(p_vec[:, np.newaxis])
    writer.writerows('\n')
    f.close()
    
    plt.figure()
    plt.plot(root[:,0],root[:,1],marker='.',markersize=12,linestyle='None')
    plt.plot(xrange,y,color='black',linestyle=':')
    plt.plot(xrange,-y,color='black',linestyle=':')
    #plt.plot(2,2,color='white',linestyle='None')
    plt.grid(which='major',color='gray',linestyle='--')
    plt.grid(which='minor',color='gray',linestyle=':')
    plt.axis('equal')
    plt.xlabel('Re')
    plt.ylabel('Im')
    #plt.xlim([-2.0,2.0])
    #plt.ylim([-2.0,2.0])
    if len(doa) == 1:
        plt.title('%d'%(doa[0,0]))
    elif len(doa) == 2:
        plt.title('%d,%d'%(doa[0,0],doa[1,0]))
    elif len(doa) == 3:
        plt.title('%d,%d,%d'%(doa[0,0],doa[1,0],doa[2,0]))
    foldername = "単位円到来波%d設定波%dランダム"%(len(doa),S_AF)
    foldername = dirname+"/"+foldername
    os.makedirs(foldername, exist_ok=True)
    if len(doa) == 1:
        plt.savefig(foldername + "/" + 'root_%d.png'%(doa[0,0]))
    elif len(doa) == 2:
        plt.savefig(foldername + "/" + 'root_%d,%d.png'%(doa[0,0],doa[1,0]))
    elif len(doa) == 3:
        plt.savefig(foldername + "/" + 'root_%d,%d,%d.png'%(doa[0,0],doa[1,0],doa[2,0]))
    plt.show()
    plt.close()
    
    #return angle,conv_array,p_normal_min
    return angle,conv_array,np.squeeze(estimated_angle),p_normal,svd_vals,xroot,xangle,xpower,num_pow,svd_vecs_vh,svd_vecs_u,z2








In [5]:
# AF w/ CS-ex(共役信号拡張)
def annihilating_1D_CSex(array):
    #source_af = numOfSource
    source_af = S_AF

    # convolution matrix
    array_length = array.shape[0]
    #checks(array_length)
    #設定波数3の場合
    if source_af == 3:
        conv_array = np.zeros((array_length-source_af-2)*(source_af+1),dtype=np.complex).reshape(array_length-source_af-2,source_af+1)
        for iter_conv in range(array_length-source_af-2):
            if iter_conv < 3:
                conv_array[iter_conv,:] = array[iter_conv:iter_conv+source_af+1][-1::-1]
            else:
                conv_array[iter_conv,:] = array[iter_conv+2:iter_conv+2+source_af+1][-1::-1]
                
    svd_vecs_u, svd_vals, svd_vecs_vh = np.linalg.svd(conv_array)
    #checks(shape(svd_vecs_u))
    checks((svd_vals))
    #checks(shape(svd_vecs_vh))
    sv = np.copy(svd_vals)
    sv /= max(sv)
    checks(sv)
    rank_CM = np.count_nonzero(sv > 0.5)
    checks(rank_CM)

    # DOA estimation
    hs_0 = np.conj(svd_vecs_vh.T)[:,-1][-1::-1]
    #checks(hs_0)
    hs = hs_0/hs_0[-1]
    #checks(hs[:])
    z = roots(hs[:-1])
    checks(z)
    #angle_response(z)

    z2 = np.zeros(source_af,dtype=np.complex)
    for iterZ in range(source_af):
        z2[iterZ] = (1/abs(z[iterZ]))*z[iterZ]
    #angle_response(z2)
    #checks(z2)
    angle = np.array(np.arcsin(np.arctan2(np.imag(z2),np.real(z2))/np.pi)*180/np.pi)[np.newaxis].T
    checks(angle)

    #filename = sSaveDataPath + "txt_annihilating_parameter.txt"
    #fout = open(filename,'w')
    # z_abs = np.abs(z)
    #fout.write("%s"%hs)
    # fout.write("\n")
    # fout.write("%s"%z_abs)
    # fout.write("\n")
    # fout.write("%s"%angle)
    # fout.write("\n")
    # fout.close()

    # power estimation
    #vand_mat = np.zeros(11*source_af,dtype=np.complex).reshape(11,source_af)
    #for iter in range(11):
    #    vand_mat[iter,:] = z**iter
    #vand_mat_inv = np.linalg.pinv(vand_mat[:])
    #checks(vand_mat)
    # checks(vand_mat_inv)
    #p_vec = (np.abs(np.dot(vand_mat_inv,array))/(numOfSample/2.))**2
    #p_normal = p_vec/np.max(p_vec)
    #p_index = np.argsort(p_vec)[-1::-1]
    #checks(p_vec[p_index])
    #checks(p_normal[p_index])
    #checks(angle[p_index])
    #p_normal_min = np.min(p_normal)/np.max(p_vec)
    #estimated_angle = angle[p_index]
    #angle = angle[p_index[0:numOfSource]]

    # power estimation
    vand_mat = np.zeros(11*source_af,dtype=np.complex).reshape(11,source_af)
    for iter in range(11):
        vand_mat[iter,:] = z2**iter
    vand_mat_inv = np.linalg.pinv(vand_mat[:])
    #checks(vand_mat)
    # checks(vand_mat_inv)
    p_vec = (np.abs(np.dot(vand_mat_inv,array))/(numOfSample/2.))**2
    checks(p_vec)
    checks(sorted(p_vec, reverse=True))
    p_normal = p_vec/np.max(p_vec)
    num_pow = p_normal
    p_index = np.argsort(p_vec)[-1::-1]
    checks(p_normal[p_index])
    p_normal = p_normal[p_index]
    checks(angle[p_index])
    xangle = angle
    """
    f = open('angle.csv', 'a', newline='')
    writer = csv.writer(f)
    writer.writerows(angle)
    writer.writerows('\n')
    f.close()
    """
    p_normal_min = np.min(p_normal)/np.max(p_vec)
    estimated_angle = angle[p_index]
    angle = angle[p_index[0:numOfSource]]

    root = np.array((np.real(z),np.imag(z))).T
    xrange = np.arange(-1,1.02,0.02)
    y = np.sqrt(1-np.around(xrange,10)**2)
    y[-1]=0
    xroot = root
    """
    f = open('root.csv', 'a', newline='')
    writer = csv.writer(f)
    writer.writerows(root)
    writer.writerows('\n')
    f.close()
    """
    print(type(p_vec))
    xpower = p_vec[:, np.newaxis]
    
    f = open('power.csv', 'a', newline='')
    writer = csv.writer(f)
    writer.writerows(p_vec[:, np.newaxis])
    writer.writerows('\n')
    f.close()
    
    plt.figure()
    plt.plot(root[:,0],root[:,1],marker='.',markersize=12,linestyle='None')
    plt.plot(xrange,y,color='black',linestyle=':')
    plt.plot(xrange,-y,color='black',linestyle=':')
    #plt.plot(2,2,color='white',linestyle='None')
    plt.grid(which='major',color='gray',linestyle='--')
    plt.grid(which='minor',color='gray',linestyle=':')
    plt.axis('equal')
    plt.xlabel('Re')
    plt.ylabel('Im')
    #plt.xlim([-2.0,2.0])
    #plt.ylim([-2.0,2.0])
    if len(doa) == 1:
        plt.title('%d'%(doa[0,0]))
    elif len(doa) == 2:
        plt.title('%d,%d'%(doa[0,0],doa[1,0]))
    elif len(doa) == 3:
        plt.title('%d,%d,%d'%(doa[0,0],doa[1,0],doa[2,0]))
    foldername = "単位円到来波%d設定波%dランダム"%(len(doa),S_AF)
    foldername = dirname+"/"+foldername
    os.makedirs(foldername, exist_ok=True)
    if len(doa) == 1:
        plt.savefig(foldername + "/" + 'root_%d.png'%(doa[0,0]))
    elif len(doa) == 2:
        plt.savefig(foldername + "/" + 'root_%d,%d.png'%(doa[0,0],doa[1,0]))
    elif len(doa) == 3:
        plt.savefig(foldername + "/" + 'root_%d,%d,%d.png'%(doa[0,0],doa[1,0],doa[2,0]))
    plt.show()
    plt.close()
    
    #return angle,conv_array,p_normal_min
    return angle,conv_array,np.squeeze(estimated_angle),p_normal,svd_vals,xroot,xangle,xpower,num_pow,svd_vecs_vh,svd_vecs_u,z2

In [6]:
if __name__ == '__main__':
    txArray = np.array([[0,1,0],[0,3,0],[0,5,0]])
    rxArray = np.array([[0,1,0],[0,2,0]])
    S_AF = 9
    numOfTx = txArray.shape[0]
    numOfRx = rxArray.shape[0]
    numOfRxData = numOfTx*numOfRx
    numOfSource = doa.shape[0]
    azAngleLen = len(azAngleArray)
    elAngleLen = len(elAngleArray)
    dt = 1/sampleFreq
    dtArray = np.arange(numOfSample)*dt
    half_ratio = lam*0.5/antDis
    
    anspro = []
    
    vroot = []
    vangle = []
    vpower = []
    vnpower = []
    doalist = []
    svdsig = []
    svdvhsig=[]
    svdusig=[]
    svdz2 = []
    estpow = []
    estang = []
    vgrad1 = []
    vgrad2 = []
    vgrad3 = []
    vgrad4 = []
    vcept1 = []
    vcept2 = []
    vcept3 = []
    vcept4 = []
    grad_err = []
    temp_arr = []
    conv_arr2to9 = []
    err_list = []
    success_point = 0
    failure_point = 0
    allroot_S = 0
    allroot_F = 0
    mulroot_S = 0
    mulroot_F = 0
    nomroot_S = 0
    nomroot_F = 0
    excroot_S = 0
    excroot_F = 0
    test = 0
    SorF_2to9 = np.zeros(9*90).reshape(9,90)
    conv_arr_2to9 = np.zeros(9*90).reshape(9,90)
    
    """
    f = open('angle.csv', 'w', newline='')
    f.close()
    f = open('root.csv', 'w', newline='')
    f.close()
    f = open('power.csv', 'w', newline='')
    f.close()
    """
    allang = np.arange(start=-90,stop = 90, step = 1)
    lock_ang = 0
    itrcnt = 100
    t_ang = 0
    
    
    #for t_ang in range(1,91):
    #for S_AFnum in range(2,11):
    S_AFnum = 3
    t_ang = 1
    s_ang = 1
    S_AF = S_AFnum
    ang1 = random.randint(-90,89)
    ang2 = random.randint(-90,89)
    while ang1 == ang2:
        ang2 = random.randint(-90,89)
    doa = np.array([[ang2,0]],dtype=float)
    doa = np.array(sorted(doa, key=lambda x:(x[0])))
    while t_ang <61:
        """
        #ang = random.randint(-90,89)
        #tang1 = -30
        #tang2 = 0

        #if tang1 <= ang:
        #    ang = ang+1
        #if tang2 <= ang:
        #    ang = ang+1
        if s_ang != t_ang:
            ang1 = random.randint(-90,89)
            ang2 = random.randint(10,70)
            while ang1 == ang2:
                ang2 = random.randint(-90,89)
            doa = np.array([[0,0][ang2,0]],dtype=float)
            doa = np.array(sorted(doa, key=lambda x:(x[0])))
            s_ang = t_ang
        """
        doa = np.array([[0,0],[t_ang,0]],dtype=float)
        #doa = np.array([[ang,0]],dtype=float)
        numOfSource = doa.shape[0]
        RandomDOA = 0
        SaveSignal = 0

        ###### Initialization ######
        totalTime = 0.0
        iterParam = 0
        paramNum = len(paramList)
        numOfRxData = txArray.shape[0]*rxArray.shape[0]
        refArray = np.zeros(numOfBlock*numOfRxData*256,dtype=np.complex).reshape(numOfBlock,numOfRxData,256)
        sigArray = np.zeros(numOfBlock*numOfRxData*256,dtype=np.complex).reshape(numOfBlock,numOfRxData,256)
        error = np.zeros(trial*doa.shape[0]*2,dtype=float).reshape(trial,doa.shape[0],2)
        angle = np.zeros(doa.shape[0]*2,dtype=float).reshape(doa.shape[0],2)
        coef = np.zeros(trial*2,dtype=float).reshape(trial,2)
        RMSEs = np.zeros(paramNum*3,dtype=float).reshape(paramNum,3)
        RMSEs_1 = np.zeros(paramNum*numOfSource*3,dtype=float).reshape(paramNum,numOfSource,3)
        corr = np.zeros(paramNum*2,dtype=np.complex)
        powers = np.zeros(paramNum,dtype=float)
        powers[:] = NaN
        p_normal = 0.
        success_NoS_estimation = np.zeros(paramNum)
        SP_AIC = np.zeros(paramNum)
        SP_MDL = np.zeros(paramNum)
        NoS_error = np.zeros(paramNum)
        succ_angle = 100*np.ones(trial*numOfSource,dtype=float).reshape(trial,numOfSource)
        fail_angle = 100*np.ones(trial*numOfSource,dtype=float).reshape(trial,numOfSource)
        Angles = np.zeros(numOfSource*trial,dtype=float).reshape(trial,numOfSource)
        estimated_numOfSource = 0


        results=np.zeros(numOfSource*trial,dtype=np.complex).reshape(trial,numOfSource)

        b=np.zeros((numOfSource+1),dtype=np.complex)
        b_sum=np.zeros(paramNum*(numOfSource+1),dtype=np.complex).reshape(paramNum,numOfSource+1)
        mVal=np.zeros(numOfRxData,dtype=np.complex)
        mVal_sum=np.zeros(numOfRxData,dtype=np.complex)
        estimated_angle = np.zeros(trial*S_AF,dtype=float).reshape(trial,S_AF)
        estimated_angle1 = np.zeros(trial*S_AF,dtype=float).reshape(trial,S_AF)
        estimated_angle2 = np.zeros(trial*S_AF,dtype=float).reshape(trial,S_AF)
        estimated_angle3 = np.zeros(trial*S_AF,dtype=float).reshape(trial,S_AF)
        estimated_angle4 = np.zeros(trial*S_AF,dtype=float).reshape(trial,S_AF)
        estimated_angle5 = np.zeros(trial*S_AF,dtype=float).reshape(trial,S_AF)
        estimated_angle6 = np.zeros(trial*S_AF,dtype=float).reshape(trial,S_AF)
        estimated_angle7 = np.zeros(trial*S_AF,dtype=float).reshape(trial,S_AF)
        estimated_angle8 = np.zeros(trial*S_AF,dtype=float).reshape(trial,S_AF)
        estimated_angle9 = np.zeros(trial*S_AF,dtype=float).reshape(trial,S_AF)
        estimated_angle10 = np.zeros(trial*S_AF,dtype=float).reshape(trial,S_AF)
        #p_normalized = zeros(numOfSource*trial,dtype=float).reshape(trial,numOfSource)
        #p_normalized_ave = np.zeros(numOfSource,dtype=float)
        p_normalized = np.zeros(trial*S_AF,dtype=float).reshape(trial,S_AF)
        p_normalized1 = np.zeros(S_AF*trial,dtype=float).reshape(trial,S_AF)
        p_normalized2 = np.zeros(S_AF*trial,dtype=float).reshape(trial,S_AF)
        p_normalized3 = np.zeros(S_AF*trial,dtype=float).reshape(trial,S_AF)
        p_normalized4 = np.zeros(S_AF*trial,dtype=float).reshape(trial,S_AF)
        p_normalized5 = np.zeros(S_AF*trial,dtype=float).reshape(trial,S_AF)
        p_normalized6 = np.zeros(S_AF*trial,dtype=float).reshape(trial,S_AF)
        p_normalized7 = np.zeros(S_AF*trial,dtype=float).reshape(trial,S_AF)
        p_normalized8 = np.zeros(S_AF*trial,dtype=float).reshape(trial,S_AF)
        p_normalized9 = np.zeros(S_AF*trial,dtype=float).reshape(trial,S_AF)
        p_normalized10 = np.zeros(S_AF*trial,dtype=float).reshape(trial,S_AF)
        p_normalized_ave = np.zeros(S_AF,dtype=float)
        sv1size = 5
        if S_AF == 9:
            sv1size = 2
        if S_AF == 8:
            sv1size = 3
        elif S_AF == 7:
            sv1size = 4
        elif S_AF == 6:
            sv1size = 5
        elif S_AF == 5:
            sv1size = 6
        elif S_AF == 4:
            sv1size = 5
        elif S_AF == 3:
            sv1size = 4
        elif S_AF == 2:
            sv1size = 3
        sv1 = np.zeros((sv1size)*trial,dtype=float).reshape(trial,sv1size)
        sv_ave1 = np.zeros(sv1size,dtype=float)
        sv2 = np.zeros((numOfSource)*trial,dtype=float).reshape(trial,numOfSource)
        sv_ave2 = np.zeros((numOfSource),dtype=float)
        Fail = 0

        # corrArray = []
        success = np.zeros(paramNum,dtype=float)
        crlbList = np.zeros(paramNum*numOfSource,dtype=np.complex).reshape(paramNum,numOfSource)
        grid_doa = np.zeros(numOfRxData*azAngleLen,dtype=np.complex).reshape(numOfRxData,azAngleLen)
        for iter in range(azAngleLen):
            grid_doa[:,iter] = ArrayWeight(azAngleArray[iter],0.0)

        ###### Simulation #######
        # for SNR in SNRList:
        for iterParam in range(paramNum):
            SNR = paramList[iterParam]

            print("============ Parameter = {0} Start ====================================\n".format(paramList[iterParam]))
            ##### initialization #####
            count = 0.0
            error[:] = 0.0
            angle = np.zeros(doa.shape[0]*2,dtype=float).reshape(doa.shape[0],2)
            angle0 = doa[np.argsort(doa[:,0])]
            delta = np.zeros(doa.shape[0]-1)
            ## debug point
            #from IPython.core.debugger import Pdb; Pdb().set_trace()

            ###### chirp interval ######
            Tm = numOfSample/sampleFreq
            BW = Tm*df
            print("chirp interval = \n{0}\n\nbandwidth = \n{1}\n".format(Tm,BW))

            ###### reference signal ######
            if not RandomDOA:
                [refArray,signalPower,corr[iterParam],crlbList[iterParam,:]] = makeSignal()
                
            ###### iteration ######
            for iterTrial in range(trial):
                print("------------ Parameter = {0} Try {1} ------------\n".format(paramList[iterParam],iterTrial+1))
                if RandomDOA:
                    while True:
                        for iterAngle in range(doa.shape[0]):
                            doa[iterAngle,0] = random.randint(0,120)-60
                        doa = np.sort(doa,axis=0)
                        #checks(doa)
                        for iterAngle in range(1,doa.shape[0]):
                            delta[iterAngle-1] = doa[iterAngle,0] - doa[iterAngle-1,0]
                        if np.all(delta>=10)==True:
                            checks(doa)
                            break
                    angle0 = doa[np.argsort(doa[:,0])]
                    Angles[iterTrial,:] = angle0[:,0]
                    ###### reference signal ######
                    [refArray,signalPower,corr[iterParam],crlbList[iterParam,:]] = makeSignal()

                ##### measurement signal (simulation)######
                [sigArray,noisePower] = addNoise(refArray,signalPower)
                if SaveSignal:
                    signame = sSaveDataPath + 'SignalData_K=' + str(numOfSource) + '_SNR=' + str(SNR) + '_DRCS=' + str(paramList[iterParam]) +'.npy'
                    np.save(signame,sigArray)

                ###### range FFT ######
                S_bin = 0
                [range_bin,range_bin_mean,fftArray,fftAmp,fftPhase,rangeProfile,peakID,bins] = range_fft(sigArray,fftSample,iRange,peakOffset,fixMode,fix_bin,fft_thr)
                    
                startTime = time.time()
                ###### DOA Estimation (preliminaries) ######
                #[KRsig,KRsig1,KRsig2,KRsig3,KRsig4,KRsig5,KRsig6,KRsig7,KRsig8,KRsig9,KRsig10,KRsig_ave] = makeKRsig(range_bin_mean)
                
                
                ###### Annihilation ######
                #sSaveDataPath = "C:\\users\\shohei\\Desktop\\Research\\FMCW\\output2020\\AF\\"
                sSaveDataPath = "C:\\Users\\benze\\Documents\\YNU\\市毛研究室\\M2森本\\AF_method\\output\\"
                #annihtest = annihilating_1D_DR(KRsig1)
                annihtest = annihilating_1D_CSex(range_bin_mean)
                
                [angle,conv_array,estimated_angle[iterTrial,:],p_normalized[iterTrial,:],sv1[iterTrial,:],xroot,xangle,xpower,x_pow,svd_vh,svd_u,zroot] = annihtest
                print("############")
                print(shape(conv_array))

                elapsedTime = time.time() - startTime
                totalTime += elapsedTime

                ##### error ######
                print('angle =\n{0}\n'.format(angle))
                if (np.all(np.isnan(angle) == True)) == True:
                    error[iterTrial,:,:] = NaN
                    print("estimation failure (nothing detected)")
                elif angle.shape[0] != numOfSource or angle.ndim == 1 or (np.any(np.isnan(angle) == True)) == True:
                    error[iterTrial,:,:] = NaN
                    print("estimation failure (mismatch # of sources)")
                else:
                    angle = angle[np.argsort(angle[:,0])]
                    # checks(angle0)
                    # checks(angle)
                    error[iterTrial,:,:] = angle - angle0
                    err_list.append(error[0])
                    if (np.any(np.isnan(angle) == True)) == True:
                        print("estimation failure (mismatch # of sources)")
                    elif np.all(np.abs(error[iterTrial,:,0]) < false_thr) == False:
                        print("estimation failure (too high error): error =\n{0}\n".format(error[iterTrial,:,:]))
                    else:
                        count += 1
                        print("estimation succeeded: error =\n{0}\n".format(error[iterTrial,:,:]))
                        RMSEs[iterParam,0:2] += np.sum((error[iterTrial,:,:])**2,axis=0)
                        RMSEs[iterParam,2] += np.sum((error[iterTrial,:,:])**2,axis=None)
                        RMSEs_1[iterParam,:,0:2] += (error[iterTrial,:,:])**2
                        RMSEs_1[iterParam,:,2] += np.sum((error[iterTrial,:,:])**2,axis=None)
                        powers[iterParam] += p_normal
                        b_sum[iterParam,:] += b[:]
                        mVal_sum[:] += mVal[:]
                        results[iterTrial,:]=angle[:,0]
                sv_ave1 += sv1[iterTrial,:]/trial
                sv_ave2 += sv2[iterTrial,:]/trial

                if estimated_numOfSource == numOfSource:
                        success_NoS_estimation[iterParam] +=1
                else:
                    NoS_error[iterParam] += (estimated_numOfSource-numOfSource)**2
                    Fail +=1
                #p_normalized_ave += p_normalized[iterTrial,:]

                        #succ_angle[iterTrial,:] = estimated_angle[:]
                #elif estimated_numOfSource != numOfSource:
                #        fail_angle[iterTrial,:] = estimated_angle[:]
                #for iterNum in range(S_AF+1):
                    #p_normalized_ave[iterNum] = sum(p_normalized[:,iterNum])/trial
                    #sv_ave[iterNum] = sum(sv[:,iterNum])/trial
            # RMSE
            if count == 0:
                RMSEs[iterParam,:] = NaN
                RMSEs_1[iterParam,:,:] = NaN
                p_normal = NaN
            else:
                RMSEs[iterParam,0:2] = np.sqrt(RMSEs[iterParam,0:2]/(numOfSource*float(count)))
                RMSEs[iterParam,2] = np.sqrt(RMSEs[iterParam,2]/(numOfSource*float(count)*2))
                RMSEs_1[iterParam,:,0:2] = np.sqrt(RMSEs_1[iterParam,:,0:2]/float(count))
                RMSEs_1[iterParam,:,2] = np.sqrt(RMSEs_1[iterParam,:,2]/(float(count)*2))
                powers[iterParam] /= float(count)
                success_NoS_estimation[iterParam] = success_NoS_estimation[iterParam]/trial
                #NoS_error[iterParam] = np.sqrt(NoS_error/float(trial))

            # Empirical CDF (azimuth)
            if trial > 1:
                #[ecdX,ecdY] = ecdf(error[:,:,0].reshape(error.shape[0]*error.shape[1],1))
                [ecdX,ecdY] = ecdf(sum(abs(error[:,:,0]),axis=1).reshape(error.shape[0],1)/numOfSource)

            #
            success[iterParam] = count
            iterParam += 1

        #date = datetime.now().strftime('%y_%m%d_%H%M_%S')
        print("============ iteration end ============")
        print("parameter list =\n{0}\n".format(paramList))
        print("successful probability =\n{0}\n".format(success/float(trial)))
        for iter_source in range(numOfSource):
            print("RMSE (Az,El) {0} =\n{1}\n".format(angle0[iter_source,:],RMSEs_1[:,iter_source,0:2]))
            # print("RMSE (both) {0} =\n{1}\n".format(angle0[iter_source,:],RMSEs_1[:,iter_source,2]))
        print("All Sources RMSE (Az,El) =\n{0}\n".format(RMSEs[:,0:2]))
        # print("All Sources RMSE (both)=\n{0}\n".format(RMSEs[:,2]))
        print("Normalized Power (Smallest Source) =\n{0}\n".format(powers))
        print("Normalized Power (Average) =\n{0}\n".format(p_normalized_ave))
        #print("b = {0}\n".format(b_sum[:,:]/float(trial)))
        #print("results = {0}\n".format(results[:,:]))
        #success_NoS_estimation[:] /= trial
        print("successful probability of numOfSource= {0}\n".format(success_NoS_estimation[:]))
        print("successful probability of numOfSource by AIC = {0}\n".format(SP_AIC))
        print("successful probability of numOfSource by MDL = {0}\n".format(SP_MDL))
        print("RMSE of estimation of numOfSource= {0}\n".format(NoS_error[:]))
        print("Average Singular Values = {0}\n".format(sv_ave1))

        print("estimation time =\n{0}\n".format(totalTime))
        print("estimation time (average)=\n{0}\n".format(totalTime/(paramNum*trial)))
        ##############################################################
        # spurious estimation
        # initial phase calculated from distance R and minimum chirp freq (fmin)
        initialPhase = 4*np.pi*(dist[1]-dist[0])*fc/3.0E8
        coefficients=spurCoefficient()
        CoefLength = coefficients.shape[0]

        # numOfsspurs
        spurList = np.zeros(CoefLength)
        #print(angle0[:,0])
        phaiList = np.pi*np.sin(np.deg2rad(doa[:,0]))
        #print(phaiList)

        # spurious list
        for iterS in range(CoefLength):
            spurList[iterS] = coefficients[iterS,0]*phaiList[0]# + coefficients[iterS,1]*phaiList[1]
        spurList*=-1

        # to keep angle in range from -pi to pi (wrap)
        spurList = (spurList+np.pi)%(2*np.pi)-np.pi
        #print(spurList)

        # to convert IF phase (phai) to arrival angle (theta)
        thetaList = np.rad2deg(np.arcsin(spurList/np.pi))
        #print(thetaList)

        
        plt.figure()
        for iterM in range(trial):
            estang.append(estimated_angle[iterM])
            estpow.append(p_normalized[iterM])
            for iterN in range(S_AF):
                plt.bar(estimated_angle[iterM,iterN],p_normalized[iterM,iterN])
        plt.vlines(doa[:,0],0,1.2,color="blue",linestyles="dashed",alpha=0.3)
        plt.yscale('log')
        plt.xlim([-90,90])
        plt.ylim(0.001,1.2)
        plt.grid(which='major',color='gray',linestyle=':')
        plt.grid(which='minor',color='gray',linestyle=':')
        plt.ylabel('Normalized Power',fontsize=15)
        plt.xlabel('Angle [deg.]',fontsize=15)
        #name = sSaveDataPath + date + '_NormPower_K=' + str(numOfSource) + '_' + str(SNR) + 'dB_P.png'
        #plt.savefig(name,bbox_inches='tight',pad_inches=0.0)
        if len(doa) == 1:
            plt.title('%d'%(doa[0,0]))
        elif len(doa) == 2:
            plt.title('%d,%d'%(doa[0,0],doa[1,0]))
        elif len(doa) == 3:
            plt.title('%d,%d,%d'%(doa[0,0],doa[1,0],doa[2,0]))
        foldername = "電力スペクトル到来波%d設定波%dランダム"%(len(doa),S_AF)
        foldername = dirname+"/"+foldername
        os.makedirs(foldername, exist_ok=True)
        if len(doa) == 1:
            plt.savefig(foldername + "/" + 'power_%d.png'%(doa[0,0]))
        elif len(doa) == 2:
            plt.savefig(foldername + "/" + 'power_%d,%d.png'%(doa[0,0],doa[1,0]))
        elif len(doa) == 3:
            plt.savefig(foldername + "/" + 'power_%d,%d,%d.png'%(doa[0,0],doa[1,0],doa[2,0]))
        plt.show()
        #plt.close
        
    ###############################################################
        #histPDF(sv1)
        if saveData:
            filename = sSaveDataPath + date + "_csv_singularValue.csv"
            fout = open(filename,'w')
            for iter in range(trial):
                fout.write("%s"%TR[iter])
                for iterp in range(5):
                    fout.write("\t")
                    fout.write("%s"%sv1[iter,iterp])
                fout.write("\n")
            fout.close()
            '''
            filename = sSaveDataPath + date + "_csv_RMSE_Azimuth_Angle=" + str(angle0.tolist()) + ".csv"
            fout = open(filename,'w')
            for iter in range(paramNum):
                fout.write("%s"%paramList[iter])
                fout.write("\t")
                fout.write("%s"%RMSEs[iter,0])
                fout.write("\n")
            fout.close()
            filename = sSaveDataPath + date + "_csv_RMSE_Elevation_Angle=" + str(angle0.tolist()) + ".csv"
            fout = open(filename,'w')
            for iter in range(paramNum):
                fout.write("%s"%paramList[iter])
                fout.write("\t")
                fout.write("%s"%RMSEs[iter,1])
                fout.write("\n")
            fout.close()
            filename = sSaveDataPath + date + "_csv_RMSE1_Azimuth_Angle=" + str(angle0.tolist()) + ".csv"
            fout = open(filename,'w')
            for iter in range(paramNum):
                fout.write("%s"%paramList[iter])
                for iterp in range(numOfSource) :
                    fout.write("\t")
                    fout.write("%s"%RMSEs_1[iter,iterp,0])
                fout.write("\n")
            fout.close()
            filename =  sSaveDataPath + date + "_csv_RMSE1_Elevation_Angle=" + str(angle0.tolist()) + ".csv"
            fout = open(filename,'w')
            for iter in range(paramNum):
                fout.write("%s"%paramList[iter])
                for iterp in range(numOfSource) :
                    fout.write("\t")
                    fout.write("%s"%RMSEs_1[iter,iterp,1])
                fout.write("\n")
            fout.close()


            hist_data = np.zeros(S_AF*trial,dtype=float).reshape(trial,S_AF)
            fits = np.zeros(S_AF*51,dtype=float).reshape(51,S_AF)
            plt.figure()
            for iterSAF in range(numOfSource-1,6):
                min_data = np.log10(min(p_normalized[:,iterSAF]))
                max_data = np.log10(max(p_normalized[:,iterSAF]))
                #checks(min_data)
                #checks(max_data)
                Lspace = np.logspace(min_data,max_data,51)
                #Lspace = np.linspace(min(p_normalized[:,iterSAF]),max(p_normalized[:,iterSAF]),101)

                hist = plt.hist(p_normalized[:,iterSAF],bins=Lspace,histtype='step',label=str(iterSAF+1))
                #checks(hist)

                n = 0
                for iterbin in range(50):
                    if hist[0][iterbin]!=0:
                        for iterbin2 in range(int(hist[0][iterbin])):
                            hist_data[n,iterSAF] = hist[1][iterbin]
                            n+=1

                #print(hist_data[:,iterSAF])
                kde = gaussian_kde(hist_data[:,iterSAF],bw_method=0.0002)
                fits[:,iterSAF] = kde(Lspace)
                #checks(fits[:,iterSAF])
                plt.plot(Lspace,fits[:,iterSAF]/max(fits[:,iterSAF])*max(hist[0][:]),label='PDF'+str(iterSAF+1))

            plt.gca().set_xscale("log")
            #xlim(1e-4,1e-2)
            #ylim(0,100)
            plt.legend(loc='upper left')
            plt.grid(which='major',color='gray',linestyle=':')
            plt.grid(which='minor',color='gray',linestyle=':')
            plt.xlabel('Normalized Power',fontsize=15)
            plt.ylabel('Number of Data',fontsize=15)
            name = sSaveDataPath + date + '_PDF_K=' + str(numOfSource) + '_' + str(SNR) + 'dB_Delta' + str(rcs) + 'dB.png'
            plt.savefig(name,bbox_inches='tight',pad_inches=0.0)
            plt.close
            plt.show()
            '''

            ###### figure output ###########################################################

            plt.rcParams["font.size"] = 12
            numOfPlots = int(fftSample/2)
            timeArray = np.arange(numOfSample)/sampleFreq
            rangeArray = np.linspace(0,sampleFreq,fftSample)*(3.0e8*numOfSample/(2*BW*sampleFreq))
            if mode_output == 1:
                # received signal (real)
                plt.figure(0,figsize=[16,10])
                timeArray = np.arange(numOfSample)/sampleFreq
                for iter in range(numOfRxData):
                    # iter = iter+2
                    pltName = "Rx " + str(iter)
                    plt.plot(timeArray,np.real(sigArray[0,iter,:]),label=pltName,color=cm.hsv(iter/float(numOfRxData)))
                    # w = signal.blackman(numOfSample)
                    # plt.plot(timeArray,np.real(w*sigArray[0,iter,:]),label=pltName,color=cm.hsv(iter/8.0))
                plt.grid()
                plt.xlabel("Time")
                plt.legend()
                titleName = "Input Signal(real)"
                plt.title(titleName)
                filename = sSaveDataPath + date + "Fig_Input Signal(real).png"
                plt.savefig(filename)
                plt.close
                # received signal (imag)
                # plt.figure(1,figsize=[16,10])
                # timeArray = np.arange(numOfSample)/sampleFreq
                # for iter in range(numOfRxData):
                #     pltName = "Rx " + str(iter)
                #     plt.plot(timeArray,np.imag(sigArray[0,iter,:]),label=pltName,color=cm.hsv(iter/float(numOfRxData)))
                #     # w = signal.blackman(numOfSample)
                #     # plt.plot(timeArray,np.imag(w*sigArray[0,iter,:]),label=pltName,color=cm.hsv(iter/8.0))
                # plt.grid()
                # plt.xlabel("Time")
                # # plt.ylabel("Power (urb unit)")
                # plt.legend()
                # titleName = "Input Signal(imag.)"
                # plt.title(titleName)
                # filename = sSaveDataPath + date + "Fig_Input Signal(imag).png"
                # plt.savefig(filename)
                # plt.close

                # corrArray
                if 'corrArray' in locals():
                    plt.figure(2,figsize=[13,5])
                    plt.subplot(1,2,1)
                    X2, Y2 = np.meshgrid(np.arange(corrArray.shape[0]+1),np.arange(corrArray.shape[0]+1))
                    pcolor(X2,Y2,np.real(corrArray[:]),cmap='jet')
                    ylim(corrArray.shape[0],0)
                    xlabel("Column")
                    ylabel("Row")
                    titlename = "Covariance Matrix (Real Part)"
                    plt.title(titlename)
                    colorbar()
                    jet()
                    plt.subplot(1,2,2)
                    X2, Y2 = np.meshgrid(np.arange(corrArray.shape[0]+1),np.arange(corrArray.shape[0]+1))
                    pcolor(X2,Y2,np.imag(corrArray[:]),cmap='jet')
                    ylim(corrArray.shape[0],0)
                    xlabel("Column")
                    ylabel("Row")
                    titlename = "Covariance Matrix (Imag Part)"
                    plt.title(titlename)
                    colorbar()
                    jet()
                    filename = sSaveDataPath + date + "_Fig_CoMatrix.png"
                    plt.savefig(filename)
                    plt.close

                # corrArray
                if 'corrArray_arg' in locals():
                    plt.figure(3,figsize=[13,5])
                    plt.subplot(1,2,1)
                    X2, Y2 = np.meshgrid(np.arange(corrArray_arg.shape[0]+1),np.arange(corrArray_arg.shape[0]+1))
                    pcolor(X2,Y2,np.real(corrArray_arg[:]),cmap='jet')
                    ylim(corrArray_arg.shape[0],0)
                    xlabel("Column")
                    ylabel("Row")
                    titlename = "Augment Covariance Matrix (Real Part)"
                    plt.title(titlename)
                    colorbar()
                    jet()
                    plt.subplot(1,2,2)
                    X2, Y2 = np.meshgrid(np.arange(corrArray_arg.shape[0]+1),np.arange(corrArray_arg.shape[0]+1))
                    pcolor(X2,Y2,np.imag(corrArray_arg[:]),cmap='jet')
                    ylim(corrArray_arg.shape[0],0)
                    xlabel("Column")
                    ylabel("Row")
                    titlename = "Augment Covariance Matrix (Imag Part)"
                    plt.title(titlename)
                    colorbar()
                    jet()
                    filename = sSaveDataPath + date + "_Fig_AugCoMatrix.png"
                    plt.savefig(filename)
                    plt.close

                # convArray
                if 'conv_array' in locals():
                    plt.figure(20,figsize=[13,5])
                    plt.subplot(1,2,1)
                    X2, Y2 = np.meshgrid(np.arange(conv_array.shape[1]+1),np.arange(conv_array.shape[0]+1))
                    pcolor(X2,Y2,np.real(conv_array[:]),cmap='jet')
                    # ylim(conv_array.shape[0],0)
                    xlabel("Column")
                    ylabel("Row")
                    titlename = "Convolution Matrix (Real Part)"
                    plt.title(titlename)
                    colorbar()
                    jet()
                    plt.subplot(1,2,2)
                    X2, Y2 = np.meshgrid(np.arange(conv_array.shape[1]+1),np.arange(conv_array.shape[0]+1))
                    pcolor(X2,Y2,np.imag(conv_array[:]),cmap='jet')
                    # ylim(conv_array.shape[0],0)
                    xlabel("Column")
                    ylabel("Row")
                    titlename = "Convolution Matrix (Imag Part)"
                    plt.title(titlename)
                    colorbar()
                    jet()
                    filename = sSaveDataPath + date + "_Fig_ConvolutionMatrix.png"
                    plt.savefig(filename)
                    plt.close

                # convArray
                if 'conv_array_0' in locals():
                    plt.figure(21,figsize=[13,5])
                    plt.subplot(1,2,1)
                    X2, Y2 = np.meshgrid(np.arange(conv_array_0.shape[1]+1),np.arange(conv_array_0.shape[0]+1))
                    pcolor(X2,Y2,np.real(conv_array_0[:]),cmap='jet')
                    # ylim(conv_array.shape[0],0)
                    xlabel("Column")
                    ylabel("Row")
                    titlename = "Convolution Matrix (Real Part)"
                    plt.title(titlename)
                    colorbar()
                    jet()
                    plt.subplot(1,2,2)
                    X2, Y2 = np.meshgrid(np.arange(conv_array_0.shape[1]+1),np.arange(conv_array_0.shape[0]+1))
                    pcolor(X2,Y2,np.imag(conv_array_0[:]),cmap='jet')
                    # ylim(conv_array.shape[0],0)
                    xlabel("Column")
                    ylabel("Row")
                    titlename = "Convolution Matrix (Imag Part)"
                    plt.title(titlename)
                    colorbar()
                    jet()
                    filename = sSaveDataPath + date + "_Fig_ConvolutionMatrix_noisy.png"
                    plt.savefig(filename)
                    plt.close

                # convArray
                if 'range_bin_mean' in locals():
                    plt.figure(21,figsize=[16,10])
                    plt.subplot(1,2,1)
                    bin_amp = np.abs(range_bin_mean)
                    plt.plot(bin_amp)
                    plt.plot(bin_amp,"ro")
                    ylim(bottom=0.0)
                    plt.grid()
                    plt.xlabel("Antenna")
                    plt.ylabel("Amplitude of Spectrum")
                    plt.title("Range Bin Amplitude")
                    plt.subplot(1,2,2)
                    bin_angle = np.rad2deg(np.arctan2(np.imag(range_bin_mean),np.real(range_bin_mean)))
                    plt.plot(bin_angle)
                    plt.plot(bin_angle,"ro")
                    ylim(-180.0,180.0)
                    plt.grid()
                    plt.xlabel("Antenna")
                    plt.ylabel("Angle of Spectrum [deg.]")
                    plt.title("Range Bin Angle")
                    filename =  sSaveDataPath + date + "Fig_range_bin.png"
                    plt.savefig(filename)
                    plt.close

                # image (Beam Former)
                if 'powerArrayBF' in locals():
                    plt.figure(3,figsize=[8,5])
                    X2, Y2 = np.meshgrid(azAngleArray,elAngleArray)
                    pcolor(X2,Y2,10*np.log10(np.abs(powerArrayBF.T)),cmap='jet',vmin=-40.0,vmax=0.0)
                    # pcolor(X2,Y2,10*np.log10(np.abs(powerArrayBF.T)),cmap='jet')
                    xlabel("Azimuth (deg)")
                    ylabel("Elevation (deg)")
                    if 'peakID' in locals():
                        titlename = "Power spectrum(Beamformer), Range=" + str(round(rangeArray[peakID[iRange]],3))
                        filename = sSaveDataPath + date + "_Fig_MIMO_3D_image_BF,Range=" + str(round(rangeArray[peakID[iRange]],3)) + ".png"
                    else:
                        titlename = "Power spectrum(Beamformer)"
                        filename = sSaveDataPath + date + "_Fig_MIMO_3D_image_BF.png"
                    plt.title(titlename)
                    colorbar()
                    plt.savefig(filename)
                    plt.close

                # image (Beam Former) 1D
                if 'powerArrayBF1D' in locals():
                    plt.figure(3,figsize=[8,5])
                    plt.plot(azAngleArray,10*np.log10(np.abs(powerArrayBF1D)))
                    xlim(-90.0,90.0)
                    # ylim(-30.0,1.0)
                    # plt.plot(azAngleArray,np.abs(powerArrayBF1D))
                    plt.xlabel("Azimuth (deg)")
                    plt.ylabel("power")
                    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(20))
                    plt.gca().xaxis.set_minor_locator(tick.MultipleLocator(5))
                    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(5))
                    plt.grid(which='major',color='gray',linestyle='--')
                    plt.grid(which='minor',color='gray',linestyle=':')
                    if 'peakID' in locals():
                        # titlename = "Power spectrum(Beamformer), Range=" + str(round(rangeArray[peakID[iRange]],3))
                        filename = sSaveDataPath + date + "_Fig_MIMO_2D_image_BF,Range=" + str(round(rangeArray[peakID[iRange]],3)) + ".png"
                    else:
                        titlename = "Power spectrum(Beamformer)"
                        filename = sSaveDataPath + date + "_Fig_MIMO_2D_image_BF.png"
                    plt.title(titlename)
                    plt.savefig(filename)
                    plt.close

                # image (2DFFT)
                if 'powerArrayFFT' in locals():
                    plt.figure(3,figsize=[8,5])
                    # azimuth_array = np.arange(powerArrayFFT.shape[0])
                    # azimuth_array = np.rad2deg(np.arcsin(np.linspace(-1,1,azNum)))
                    azimuth_array = np.rad2deg(np.arcsin(np.linspace(-half_ratio,half_ratio,azNum)))
                    range_array = np.linspace(0,sampleFreq,rangeNum)*(3.0e8*rangeNum/(2*BW*sampleFreq))
                    X2, Y2 = np.meshgrid(azimuth_array,range_array)
                    # pcolor(X2,Y2,10*np.log10(np.abs(powerArrayFFT.T)/np.max(np.abs(powerArrayFFT))),cmap='jet')
                    pcolor(X2,Y2,np.abs(powerArrayFFT.T),cmap='jet')
                    plot(azimuth_array[peakIndex[:,0]],range_array[peakIndex[:,1]],'w.')
                    # pcolor(X2,Y2,10*np.log10(np.abs(powerArrayBF.T)),cmap='jet')
                    xlabel("Azimuth (deg)")
                    ylabel("Range (m)")
                    titlename = "Power spectrum (JointFFT)"
                    filename = sSaveDataPath + date + "_Fig_MIMO_JointFFT.png"
                    plt.title(titlename)
                    colorbar()
                    plt.savefig(filename)
                    plt.close

                # image (Music)
                if 'powerArrayMusic' in locals():
                    plt.figure(4,figsize=[8,5])
                    X2, Y2 = np.meshgrid(azAngleArray,elAngleArray)
                    pcolor(X2,Y2,10*log10(np.abs(powerArrayMusic.T)),cmap='jet',vmin=-30.0,vmax=0.0)
                    xlabel("Azimuth (deg)")
                    ylabel("Elevation (deg)")
                    if 'peakID' in locals():
                        titlename = "MUSIC spectrum, Range=" + str(round(rangeArray[peakID[iRange]],3))
                        filename = sSaveDataPath + date + "_Fig_MIMO_3D_image_Music,Range=" + str(round(rangeArray[peakID[iRange]],3)) + ".png"
                    else:
                        titlename = "Power spectrum (MUSIC)"
                        filename = sSaveDataPath + date + "_Fig_MIMO_3D_image_Music.png"
                    plt.title(titlename)
                    colorbar()
                    jet()
                    plt.savefig(filename)
                    plt.close

                # image (Music) 1D
                if 'powerArrayMusic1D' in locals():
                    plt.figure(4,figsize=[8,5])
                    plot(azAngleArray,10*np.log10(np.abs(powerArrayMusic1D)),'g',label="MUSIC")
                    # plot(azAngleArray,10*np.log10(np.abs(powerArrayMusic1Dssp)),'r',label="MUSIC-SS3")
                    plt.legend()
                    # plot(azAngleArray,np.abs(powerArrayMusic1D))
                    xlim(-90.0,90.0)
                    xlabel("Azimuth (deg)")
                    ylabel("MUSIC Spectrum")
                    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(20))
                    plt.gca().xaxis.set_minor_locator(tick.MultipleLocator(5))
                    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(5))
                    plt.grid(which='major',color='gray',linestyle='--')
                    plt.grid(which='minor',color='gray',linestyle=':')
                    if 'peakID' in locals():
                        titlename = "Power spectrum(MUSIC), Range=" + str(round(rangeArray[peakID[iRange]],3))
                        filename = sSaveDataPath + date + "_Fig_MIMO_2D_image_MUSIC,Range=" + str(round(rangeArray[peakID[iRange]],3)) + ".png"
                    else:
                        titlename = "Power spectrum (MUSIC)"
                        filename = sSaveDataPath + date + "_Fig_MIMO_2D_image_MUSIC.png"
                    # plt.title(titlename)
                    plt.savefig(filename)
                    plt.close

                # image (Orthogonal Matching Pursuit) 1D
                if 'powerArrayOMP' in locals():
                    plt.figure(4,figsize=[8,5])
                    plot(azAngleArray,np.abs(powerArrayOMP))
                    xlabel("Azimuth (deg)")
                    ylabel("OMP Result")
                    if 'peakID' in locals():
                        titlename = "Power spectrum (OMP), Range=" + str(round(rangeArray[peakID[iRange]],3))
                        filename = sSaveDataPath + date + "_Fig_MIMO_2D_image_OMP,Range=" + str(round(rangeArray[peakID[iRange]],3)) + ".png"
                    else:
                        titlename = "Power spectrum (OMP)"
                        filename = sSaveDataPath + date + "_Fig_MIMO_2D_image_OMP.png"
                    plt.title(titlename)
                    plt.savefig(filename)
                    plt.close

                # Spectrum(Amplitude)
                if 'peakID' in locals():
                    plt.figure(5,figsize=[16,10])
                    for iter in range(numOfRxData):
                        pltName = "No." + str(iter)
                        plt.plot(rangeArray[0:numOfPlots],fftAmp[iter,0:numOfPlots],label=pltName,color=cm.hsv(iter/numOfRxData))
                    # plt.plot(rangeArray[peakID],fftAmp[0,0,peakID],"ro",label='peak')
                    plt.plot(rangeArray[0:numOfPlots],rangeProfile[0:numOfPlots],label="Range Profile",color="k")
                    # plt.ylim([0,50])
                    plt.grid()
                    plt.xlabel("Range")
                    plt.ylabel("Normalized Power")
                    plt.legend()
                    plt.title("Range Spectrum")
                    filename = sSaveDataPath + date + "_Fig_Spectrum(Amplitude).png"
                    plt.savefig(filename)
                    plt.close

                # Spectrum(Phase)
                # if 'peakID' in locals():
                #     plt.figure(6,figsize=[16,10])
                #     for iter in range(1):
                #         iter += 7
                #         pltName = "No." + str(iter)
                #         plt.plot(rangeArray[0:numOfPlots],fftPhase[iter,0:numOfPlots],label=pltName,color=cm.hsv(iter/8.0))
                #         plt.plot(rangeArray[0:numOfPlots],fftPhase[iter,0:numOfPlots],"ko",label=pltName)
                #     plt.grid()
                #     plt.xlabel("Range")
                #     plt.ylabel("Phase")
                #     plt.legend()
                #     plt.title("Range Spectrum")
                #     filename =  sSaveDataPath + date + "Fig_Spectrum(Phase).png"
                #     plt.savefig(filename)
                #     plt.close

                # Spectrum(Complex)
                # if 'peakID' in locals():
                #     fftReal = np.real(fftArray)
                #     fftImag = np.imag(fftArray)
                #     plt.figure(7,figsize=[16,10])
                #     for iter in range(3):
                #         pltName1 = "No." + str(iter) + "(real)"
                #         pltName2 = "No." + str(iter) + "(imag)"
                #         plt.plot(rangeArray[0:numOfPlots],fftReal[iter,0:numOfPlots],label=pltName1,color=cm.hsv(iter/8.0))
                #         plt.plot(rangeArray[0:numOfPlots],fftImag[iter,0:numOfPlots],label=pltName2,color=cm.hsv(iter/8.0+0.5))
                #     plt.grid()
                #     plt.xlabel("Range")
                #     plt.ylabel("Phase")
                #     plt.legend()
                #     plt.title("Range Spectrum")
                #     filename = sSaveDataPath + date + "Fig_Spectrum(Complex).png"
                #     plt.savefig(filename)
                #     plt.close

                # Extracted Spectrum (Amplitude)
                if 'fftArray2' in locals():
                    plt.figure(8,figsize=[16,10])
                    for iter in range(8):
                        pltName = "No." + str(iter)
                        plt.plot(rangeArray[0:numOfPlots],10*log10(np.abs(fftArray2[iter,0:numOfPlots])),label=pltName,color=cm.hsv(iter/8.0))
                    # plt.plot(rangeArray[peakID],fftAmp[0,0,peakID],"ro",label='peak')
                    xlim(rangeArray[0],rangeArray[numOfPlots])
                    plt.grid()
                    plt.xlabel("Range")
                    plt.ylabel("Normalized Power")
                    plt.legend()
                    plt.title("Extracted Range Spectrum")
                    filename = sSaveDataPath + date + "_Fig_Extracted_Spectrum(Amplitude).png"
                    plt.savefig(filename)
                    plt.close

                # corrArray (min rank)
                if 'corrArray_opt' in locals():
                    plt.figure(9,figsize=[13,5])
                    plt.subplot(1,2,1)
                    X2, Y2 = np.meshgrid(np.arange(corrArray_opt.shape[0]+1),np.arange(corrArray_opt.shape[0]+1))
                    pcolor(X2,Y2,np.real(corrArray_opt[:]),cmap='jet')
                    ylim(corrArray_opt.shape[0],0)
                    xlabel("Column")
                    ylabel("Row")
                    titlename = "Opt Covariance Matrix (Real Part)"
                    plt.title(titlename)
                    colorbar()
                    jet()
                    plt.subplot(1,2,2)
                    X2, Y2 = np.meshgrid(np.arange(corrArray_opt.shape[0]+1),np.arange(corrArray_opt.shape[0]+1))
                    pcolor(X2,Y2,np.imag(corrArray_opt[:]),cmap='jet')
                    ylim(corrArray_opt.shape[0],0)
                    xlabel("Column")
                    ylabel("Row")
                    titlename = "Opt Covariance Matrix (Imag Part)"
                    plt.title(titlename)
                    colorbar()
                    jet()
                    filename = sSaveDataPath + date + "_Fig_CoMatrix_Opt.png"
                    plt.savefig(filename)
                    plt.close

                ###### text output #############################################################
                # filename = sSaveDataPath + "txt_CRLB_angleDef.txt"
                # fout = open(filename,'w')
                # for iter in range(paramNum):
                #     crlb = np.rad2deg(np.sqrt(6/(numOfSample*(10**(SNR/10))*numOfRxData*(numOfRxData**2-1)*(2*np.pi/lam*antDis*np.cos(deg2rad(paramList[iter]/2)))**2)))
                #     fout.write("%s"%paramList[iter])
                #     fout.write("\t")
                #     fout.write("%s"%crlb)
                #     fout.write("\n")
                # fout.close()
                filename = sSaveDataPath + date + "_txt_ParameterList.txt"
                fout = open(filename,'w')
                for iter in range(paramNum):
                    fout.write("%s"%paramList[iter])
                    fout.write("\n")
                fout.close()
                filename = sSaveDataPath + date + "_txt_CRLB_" + str(doa.tolist()) +".txt"
                fout = open(filename,'w')
                for iter in range(paramNum):
                    fout.write("%s"%paramList[iter])
                    for iterp in range(numOfSource) :
                        fout.write("\t")
                        fout.write("%s"%np.real(crlbList[iter,iterp]))
                    fout.write("\n")
                fout.close()

                if trial > 1:
                    filename = sSaveDataPath + date + "_csv_SuccessfulProbability.csv"
                    fout = open(filename,'w')
                    prob = 0
                    for iter in range(paramNum):
                        prob = success[iter]/float(trial)
                        fout.write("%s"%paramList[iter])
                        fout.write("\t")
                        fout.write("%s"%prob)
                        fout.write("\n")
                    fout.close()
                    filename = sSaveDataPath + date + "_csv_RMSE_Azimuth_Angle=" + str(angle0.tolist()) + ".csv"
                    fout = open(filename,'w')
                    for iter in range(paramNum):
                        fout.write("%s"%paramList[iter])
                        fout.write("\t")
                        fout.write("%s"%RMSEs[iter,0])
                        fout.write("\n")
                    fout.close()
                    filename = sSaveDataPath + date + "_csv_RMSE_Elevation_Angle=" + str(angle0.tolist()) + ".csv"
                    fout = open(filename,'w')
                    for iter in range(paramNum):
                        fout.write("%s"%paramList[iter])
                        fout.write("\t")
                        fout.write("%s"%RMSEs[iter,1])
                        fout.write("\n")
                    fout.close()
                    filename = sSaveDataPath + date + "_csv_RMSE1_Azimuth_Angle=" + str(angle0.tolist()) + ".csv"
                    fout = open(filename,'w')
                    for iter in range(paramNum):
                        fout.write("%s"%paramList[iter])
                        for iterp in range(numOfSource) :
                            fout.write("\t")
                            fout.write("%s"%RMSEs_1[iter,iterp,0])
                        fout.write("\n")
                    fout.close()
                    filename = sSaveDataPath + date + "_csv_RMSE1_Elevation_Angle=" + str(angle0.tolist()) + ".csv"
                    fout = open(filename,'w')
                    for iter in range(paramNum):
                        fout.write("%s"%paramList[iter])
                        for iterp in range(numOfSource) :
                            fout.write("\t")
                            fout.write("%s"%RMSEs_1[iter,iterp,1])
                        fout.write("\n")
                    fout.close()
                    filename = sSaveDataPath + date + "_csv_ECDF_Azimuth_Angle=" + str(angle0.tolist()) + ".csv"
                    fout = open(filename,'w')
                    for iter in range(len(ecdY)):
                        fout.write("%s"%ecdX[iter])
                        fout.write("\t")
                        fout.write("%s"%ecdY[iter])
                        fout.write("\n")
                    fout.close()
                    filename = sSaveDataPath + date + "_csv_EachAngle_Azimuth_Angle=" + str(angle0.tolist()) + ".csv"
                    fout = open(filename,'w')
                    for iter in range(trial):
                        fout.write("%s"%TR[iter])
                        for iterp in range(numOfSource):
                            fout.write("\t")
                            fout.write("%s"%np.real(results[iter,iterp]))
                        fout.write("\n")
                    fout.close()

                    filename = sSaveDataPath + date + "_csv_SuccessfulProbability_numofsource.csv"
                    fout = open(filename,'w')
                    for iter in range(paramNum):
                        fout.write("%s"%paramList[iter])
                        fout.write("\t")
                        fout.write("%s"%success_NoS_estimation[iter])
                        fout.write("\n")
                    fout.close()

                    filename = sSaveDataPath + date + "_csv_SuccessfulProbability_numofsource_AIC.csv"
                    fout = open(filename,'w')
                    for iter in range(paramNum):
                        fout.write("%s"%paramList[iter])
                        fout.write("\t")
                        fout.write("%s"%SP_AIC[iter])
                        fout.write("\n")
                    fout.close()

                    filename = sSaveDataPath + date + "_csv_SuccessfulProbability_numofsource_MDL.csv"
                    fout = open(filename,'w')
                    for iter in range(paramNum):
                        fout.write("%s"%paramList[iter])
                        fout.write("\t")
                        fout.write("%s"%SP_MDL[iter])
                        fout.write("\n")
                    fout.close()
                    filename = sSaveDataPath + date + "_csv_EachAngle_Azimuth_Error=" + str(angle0.tolist()) + ".csv"
                    fout = open(filename,'w')
                    for iter in range(trial):
                        fout.write("%s"%TR[iter])
                        for iterp in range(numOfSource):
                            fout.write("\t")
                            fout.write("%s"%np.real(abs(error[iter,iterp,0])))
                        fout.write("\n")
                    fout.close()
                    filename = sSaveDataPath + date + "_csv_EachAngle_Azimuth_ErrorAve=" + str(angle0.tolist()) + ".csv"
                    fout = open(filename,'w')
                    for iter in range(trial):
                        fout.write("%s"%TR[iter])
                        fout.write("\t")
                        ave=np.sum(abs(error[iter,:,0]),dtype=float)/float(numOfSource)
                        fout.write("%s"%ave)
                        fout.write("\n")
                    fout.close()
                    filename = sSaveDataPath + date + "_csv_EachAngle_Azimuth_of_Arrival=" + str(angle0.tolist()) + ".csv"
                    fout = open(filename,'w')
                    for iter in range(trial):
                        fout.write("%s"%TR[iter])
                        for iterp in range(numOfSource):
                            fout.write("\t")
                            fout.write("%s"%Angles[iter,iterp])
                        fout.write("\n")
                    fout.close()


                    filename = sSaveDataPath + date + "_csv_EachAngle_estimated_Angle_succes=" + str(angle0.tolist()) + ".csv"
                    fout = open(filename,'w')
                    for iter in range(trial):
                        fout.write("%s"%TR[iter])
                        for iterp in range(numOfSource):
                            fout.write("\t")
                            fout.write("%s"%(succ_angle[iter,iterp]))
                        fout.write("\n")
                    fout.close()
                    filename = sSaveDataPath + date + "_csv_EachAngle_estimated_Angle_fail=" + str(angle0.tolist()) + ".csv"
                    fout = open(filename,'w')
                    for iter in range(trial):
                        fout.write("%s"%TR[iter])
                        for iterp in range(numOfSource):
                            fout.write("\t")
                            fout.write("%s"%(fail_angle[iter,iterp]))
                        fout.write("\n")
                    fout.close()


        print("============ [End] {0} ============".format(date))


        if S_AF == 7:
            grad1 = (np.conj(svd_vh.T)[0,S_AF].imag - np.conj(svd_vh.T)[7,S_AF].imag) / (np.conj(svd_vh.T)[0,S_AF].real - np.conj(svd_vh.T)[7,S_AF].real)
            vgrad1.append(math.degrees(math.atan(grad1)))
            grad2 = (np.conj(svd_vh.T)[1,S_AF].imag - np.conj(svd_vh.T)[6,S_AF].imag) / (np.conj(svd_vh.T)[1,S_AF].real - np.conj(svd_vh.T)[6,S_AF].real)
            vgrad2.append(math.degrees(math.atan(grad2)))
            grad3 = (np.conj(svd_vh.T)[2,S_AF].imag - np.conj(svd_vh.T)[5,S_AF].imag) / (np.conj(svd_vh.T)[2,S_AF].real - np.conj(svd_vh.T)[5,S_AF].real)
            vgrad3.append(math.degrees(math.atan(grad3)))
            grad4 = (np.conj(svd_vh.T)[3,S_AF].imag - np.conj(svd_vh.T)[4,S_AF].imag) / (np.conj(svd_vh.T)[3,S_AF].real - np.conj(svd_vh.T)[4,S_AF].real)
            vgrad4.append(math.degrees(math.atan(grad4)))            
            cept1 = (np.conj(svd_vh.T)[0,S_AF].imag + np.conj(svd_vh.T)[7,S_AF].imag) / 2 - (-1/grad1) *  (np.conj(svd_vh.T)[0,S_AF].real + np.conj(svd_vh.T)[7,S_AF].real) / 2
            vcept1.append(vcept1)
            cept2 = (np.conj(svd_vh.T)[1,S_AF].imag + np.conj(svd_vh.T)[6,S_AF].imag) / 2 - (-1/grad2) *  (np.conj(svd_vh.T)[1,S_AF].real + np.conj(svd_vh.T)[6,S_AF].real) / 2
            vcept2.append(vcept2)
            cept3 = (np.conj(svd_vh.T)[2,S_AF].imag + np.conj(svd_vh.T)[5,S_AF].imag) / 2 - (-1/grad3) *  (np.conj(svd_vh.T)[2,S_AF].real + np.conj(svd_vh.T)[5,S_AF].real) / 2
            vcept3.append(vcept3)
            cept4 = (np.conj(svd_vh.T)[3,S_AF].imag + np.conj(svd_vh.T)[4,S_AF].imag) / 2 - (-1/grad4) *  (np.conj(svd_vh.T)[3,S_AF].real + np.conj(svd_vh.T)[4,S_AF].real) / 2
            vcept4.append(vcept4)            
        elif S_AF == 6:
            grad1 = (np.conj(svd_vh.T)[0,S_AF].imag - np.conj(svd_vh.T)[6,S_AF].imag) / (np.conj(svd_vh.T)[0,S_AF].real - np.conj(svd_vh.T)[6,S_AF].real)
            vgrad1.append(math.degrees(math.atan(grad1)))
            grad2 = (np.conj(svd_vh.T)[1,S_AF].imag - np.conj(svd_vh.T)[5,S_AF].imag) / (np.conj(svd_vh.T)[1,S_AF].real - np.conj(svd_vh.T)[5,S_AF].real)
            vgrad2.append(math.degrees(math.atan(grad2)))
            grad3 = (np.conj(svd_vh.T)[2,S_AF].imag - np.conj(svd_vh.T)[4,S_AF].imag) / (np.conj(svd_vh.T)[2,S_AF].real - np.conj(svd_vh.T)[4,S_AF].real)
            vgrad3.append(math.degrees(math.atan(grad3)))
            cept1 = (np.conj(svd_vh.T)[0,S_AF].imag + np.conj(svd_vh.T)[6,S_AF].imag) / 2 - (-1/grad1) *  (np.conj(svd_vh.T)[0,S_AF].real + np.conj(svd_vh.T)[6,S_AF].real) / 2
            vcept1.append(vcept1)
            cept2 = (np.conj(svd_vh.T)[1,S_AF].imag + np.conj(svd_vh.T)[5,S_AF].imag) / 2 - (-1/grad2) *  (np.conj(svd_vh.T)[1,S_AF].real + np.conj(svd_vh.T)[5,S_AF].real) / 2
            vcept2.append(vcept2)
            cept3 = (np.conj(svd_vh.T)[2,S_AF].imag + np.conj(svd_vh.T)[4,S_AF].imag) / 2 - (-1/grad3) *  (np.conj(svd_vh.T)[2,S_AF].real + np.conj(svd_vh.T)[4,S_AF].real) / 2
            vcept3.append(vcept3)
        elif S_AF == 5:
            grad1 = (np.conj(svd_vh.T)[0,S_AF].imag - np.conj(svd_vh.T)[5,S_AF].imag) / (np.conj(svd_vh.T)[0,S_AF].real - np.conj(svd_vh.T)[5,S_AF].real)
            vgrad1.append(math.degrees(math.atan(grad1)))
            grad2 = (np.conj(svd_vh.T)[1,S_AF].imag - np.conj(svd_vh.T)[4,S_AF].imag) / (np.conj(svd_vh.T)[1,S_AF].real - np.conj(svd_vh.T)[4,S_AF].real)
            vgrad2.append(math.degrees(math.atan(grad2)))
            grad3 = (np.conj(svd_vh.T)[2,S_AF].imag - np.conj(svd_vh.T)[3,S_AF].imag) / (np.conj(svd_vh.T)[2,S_AF].real - np.conj(svd_vh.T)[3,S_AF].real)
            vgrad3.append(math.degrees(math.atan(grad3)))
            cept1 = (np.conj(svd_vh.T)[0,S_AF].imag + np.conj(svd_vh.T)[5,S_AF].imag) / 2 - (-1/grad1) *  (np.conj(svd_vh.T)[0,S_AF].real + np.conj(svd_vh.T)[5,S_AF].real) / 2
            cept2 = (np.conj(svd_vh.T)[1,S_AF].imag + np.conj(svd_vh.T)[4,S_AF].imag) / 2 - (-1/grad2) *  (np.conj(svd_vh.T)[1,S_AF].real + np.conj(svd_vh.T)[4,S_AF].real) / 2
            cept3 = (np.conj(svd_vh.T)[2,S_AF].imag + np.conj(svd_vh.T)[3,S_AF].imag) / 2 - (-1/grad3) *  (np.conj(svd_vh.T)[2,S_AF].real + np.conj(svd_vh.T)[3,S_AF].real) / 2            
        elif S_AF == 4:
            grad1 = (np.conj(svd_vh.T)[0,S_AF].imag - np.conj(svd_vh.T)[4,S_AF].imag) / (np.conj(svd_vh.T)[0,S_AF].real - np.conj(svd_vh.T)[4,S_AF].real)
            grad2 = (np.conj(svd_vh.T)[1,S_AF].imag - np.conj(svd_vh.T)[3,S_AF].imag) / (np.conj(svd_vh.T)[1,S_AF].real - np.conj(svd_vh.T)[3,S_AF].real)
            cept1 = (np.conj(svd_vh.T)[0,S_AF].imag + np.conj(svd_vh.T)[4,S_AF].imag) / 2 - (-1/grad1) *  (np.conj(svd_vh.T)[0,S_AF].real + np.conj(svd_vh.T)[4,S_AF].real) / 2
            cept2 = (np.conj(svd_vh.T)[1,S_AF].imag + np.conj(svd_vh.T)[3,S_AF].imag) / 2 - (-1/grad2) *  (np.conj(svd_vh.T)[1,S_AF].real + np.conj(svd_vh.T)[3,S_AF].real) / 2            
        elif S_AF == 3:
            grad1 = (np.conj(svd_vh.T)[0,S_AF].imag - np.conj(svd_vh.T)[3,S_AF].imag) / (np.conj(svd_vh.T)[0,S_AF].real - np.conj(svd_vh.T)[3,S_AF].real)
            grad2 = (np.conj(svd_vh.T)[1,S_AF].imag - np.conj(svd_vh.T)[2,S_AF].imag) / (np.conj(svd_vh.T)[1,S_AF].real - np.conj(svd_vh.T)[2,S_AF].real)
            cept1 = (np.conj(svd_vh.T)[0,S_AF].imag + np.conj(svd_vh.T)[3,S_AF].imag) / 2 - (-1/grad1) *  (np.conj(svd_vh.T)[0,S_AF].real + np.conj(svd_vh.T)[3,S_AF].real) / 2
            cept2 = (np.conj(svd_vh.T)[1,S_AF].imag + np.conj(svd_vh.T)[2,S_AF].imag) / 2 - (-1/grad2) *  (np.conj(svd_vh.T)[1,S_AF].real + np.conj(svd_vh.T)[2,S_AF].real) / 2
        elif S_AF == 2:
            grad1 = (np.conj(svd_vh.T)[0,S_AF].imag - np.conj(svd_vh.T)[2,S_AF].imag) / (np.conj(svd_vh.T)[0,S_AF].real - np.conj(svd_vh.T)[2,S_AF].real)
            cept1 = (np.conj(svd_vh.T)[0,S_AF].imag + np.conj(svd_vh.T)[2,S_AF].imag) / 2 - (-1/grad1) *  (np.conj(svd_vh.T)[0,S_AF].real + np.conj(svd_vh.T)[2,S_AF].real) / 2
        xrange = np.arange(-1,1.02,0.02)
        if S_AF == 7:
            y_grad1 = xrange*(-1/grad1) + cept1
            y_grad2 = xrange*(-1/grad2) + cept2
            y_grad3 = xrange*(-1/grad3) + cept3
            y_grad4 = xrange*(-1/grad4) + cept4
        elif S_AF == 6 or S_AF == 5:
            y_grad1 = xrange*(-1/grad1) + cept1
            y_grad2 = xrange*(-1/grad2) + cept2
            y_grad3 = xrange*(-1/grad3) + cept3
        elif S_AF == 4 or S_AF == 3:
            y_grad1 = xrange*(-1/grad1) + cept1
            y_grad2 = xrange*(-1/grad2) + cept2
        elif S_AF == 2:
            y_grad1 = xrange*(-1/grad1) + cept1

        y = np.sqrt(1-np.around(xrange,10)**2)
        y[-1]=0

        ####estimation success?####
        print(sort(angle.T)[0])
        print(sort(doa[:,0]))

        temp_arr.append(conv_array)
        vroot.append(annihtest[5])
        vangle.append(annihtest[6])
        vpower.append(annihtest[7])
        vnpower.append(annihtest[8])
        doalist.append([doa[0,0],doa[0,1]])
        svdsig.append(sv1[0])
        svdvhsig.append(svd_vh)
        svdusig.append(svd_u)
        svdz2.append(zroot)
        print(S_AF)
        print("############################################################################")
        t_ang=t_ang+1
       
        
    #conv_arr2to9.append(temp_arr)
    #temp_arr = []

============ Parameter = 20.0 Start ====================================

chirp interval = 
4.654545454545455e-05

bandwidth = 
3160000000.0

DOA = [0.0 , 0.0]
u = [-120.543125 -120.543125 -120.543125 -120.543125 -120.543125 -120.543125]

phase shift = [-120.543125 -120.543125 -120.543125 -120.543125 -120.543125 -120.543125]

DOA = [1.0 , 0.0]
u = [-114.26025868 -111.11882552 -107.97739237 -104.83595921 -101.69452605
  -98.55309289]

phase shift = [-114.26025868 -111.11882552 -107.97739237 -104.83595921 -101.69452605
  -98.55309289]

beat frequency=
[452604.16666667 452604.16666667]

beat interval=
[2.20943613e-06 2.20943613e-06]

signal covariance =
[[1.-6.5200143e-19j 1.-6.5200143e-19j]
 [1.-6.5200143e-19j 1.-6.5200143e-19j]]

signal covariance (Abs)=
[[1. 1.]
 [1. 1.]]

signal covariance (Arg)=
[[-3.73569302e-17 -3.73569302e-17]
 [-3.73569302e-17 -3.73569302e-17]]

signal power=
[1.-6.5200143e-19j 1.-6.5200143e-19j]

------------ Parameter = 20.0 Try 1 ------------

Noise Power=
0.0

ValueError: shapes (3,11) and (6,) not aligned: 11 (dim 1) != 6 (dim 0)